# Level One :

**Task One : Data preprocessing for Machine Learning**

  * **Description**: Preprocess a raw dataset to make it ready for machine learning.

**Objective**:
  * Handle missing data (e.g., filling with mean/median,dropping).
  * Encode categorical variables (e.g. using one-hot encoding or label encoding).
  * Normalize or standardize numerical feaures.
  * Split the dataset into training and testing sets.
  * Tools: Python, pandas, sckikit-learn.



## **0.1 Connect to google colab**

In [454]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [455]:
import os
os.chdir('/content/drive/MyDrive/Codveda Technologies')

## **0.2 Setup the device**

In [456]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## **0.3 Import Dependencies**

In [457]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from rich.console import Console
from rich.table import Table
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.box import ROUNDED
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from rich.console import Console
from rich.table import Table
import pickle
from pathlib import Path
console = Console()


## **1.1 Load dataset**

In [458]:
# Load Iris dataset
file_path_iris = "/content/drive/MyDrive/Codveda Technologies/data/1) iris.csv"
df_iris = pd.read_csv(file_path_iris)
# Load House dataset
file_path_house = "/content/drive/MyDrive/Codveda Technologies/data/4) house Prediction Data Set.csv"
df_house = pd.read_csv(file_path_house)
# Load Stock dataset
file_path_stock = "/content/drive/MyDrive/Codveda Technologies/data/2) Stock Prices Data Set.csv"
df_stock = pd.read_csv(file_path_stock)
# Load Sentiment dataset
file_path_sentiment = "/content/drive/MyDrive/Codveda Technologies/data/3) Sentiment dataset.csv"
df_sentiment = pd.read_csv(file_path_sentiment)

file_path_churn_20 = "/content/drive/MyDrive/Codveda Technologies/data/Churn Prdiction Data/churn-bigml-20.csv"
df_churn_20 = pd.read_csv(file_path_churn_20)

file_path_churn_80 = "/content/drive/MyDrive/Codveda Technologies/data/Churn Prdiction Data/churn-bigml-80.csv"
df_churn_80 = pd.read_csv(file_path_churn_80)

## **1.2 Dataset Overview**

In [459]:
def dataset_report(df: pd.DataFrame, file_path: str):
    """Generate a stunning dataset report using Rich tables and panels."""
    file_name = os.path.basename(file_path)
    file_size = os.path.getsize(file_path) / 1024  # KB

    # --- Header ---
    console.print(Panel.fit(
        f"[bold cyan]📊 Dataset Report[/bold cyan]\n"
        f"[white]📂 {file_name}[/white]  |  [green]{df.shape[0]} rows[/green] × [green]{df.shape[1]} cols[/green]  |  [yellow]{file_size:.2f} KB[/yellow]",
        border_style="cyan", box=ROUNDED))

    # --- Columns Overview ---
    t_cols = Table(title="🧾 Columns Overview", box=ROUNDED, show_lines=True)
    t_cols.add_column("Column", style="bold magenta")
    t_cols.add_column("Type")
    t_cols.add_column("Non-Null Count")
    t_cols.add_column("Missing Values")
    for col in df.columns:
        t_cols.add_row(
            col,
            str(df[col].dtype),
            str(df[col].notna().sum()),
            str(df[col].isna().sum())
        )
    console.print(t_cols)

    # --- Column Type Summary ---
    num_cols = df.select_dtypes(include=[np.number]).columns
    cat_cols = df.select_dtypes(exclude=[np.number]).columns
    console.print(Panel.fit(
        f"[cyan]🔢 Numerical:[/cyan] {len(num_cols)} → {', '.join(num_cols[:10])}{' ...' if len(num_cols) > 10 else ''}\n"
        f"[yellow]🔤 Categorical:[/yellow] {len(cat_cols)} → {', '.join(cat_cols[:10])}{' ...' if len(cat_cols) > 10 else ''}",
        title="Column Types", border_style="magenta", box=ROUNDED))

    # --- Missing Values ---
    miss = df.isna().sum()
    if miss.sum() > 0:
        t_miss = Table(title="⚠ Missing Values", box=ROUNDED, show_lines=True)
        t_miss.add_column("Column", style="bold cyan")
        t_miss.add_column("Missing Count")
        for c, v in miss[miss > 0].sort_values(ascending=False).items():
            t_miss.add_row(c, str(int(v)))
        console.print(t_miss)
    else:
        console.print(Panel.fit("[green]✓ No Missing Values[/green]", border_style="green", box=ROUNDED))

    # --- Duplicates ---
    dups = df.duplicated().sum()
    if dups > 0:
        console.print(Panel.fit(f"[red]✗ Duplicated Rows:[/red] {dups}", border_style="red", box=ROUNDED))
    else:
        console.print(Panel.fit("[green]✓ No Duplicated Rows[/green]", border_style="green", box=ROUNDED))

    # --- Descriptive Stats ---
    desc = df.describe(include="all", percentiles=[.25, .5, .75]).T.fillna("")
    t_desc = Table(title="📈 Descriptive Statistics", box=ROUNDED, show_lines=True)
    t_desc.add_column("Column", style="bold magenta")
    for col in desc.columns:
        t_desc.add_column(str(col))
    for idx, row in desc.iterrows():
        t_desc.add_row(str(idx), *[str(val) for val in row.values])
    console.print(t_desc)

    # --- First 5 Examples ---
    t_head = Table(title="📝 First 5 Rows", box=ROUNDED, show_lines=True)
    for col in df.columns:
        t_head.add_column(str(col), overflow="fold", style="cyan")
    for _, row in df.head(5).iterrows():
        t_head.add_row(*[str(val) for val in row.values])
    console.print(t_head)


In [460]:
dataset_report(df_iris, file_path_iris)

╭──────────────────────────────────────────────────╮
│ 📊 Dataset Report                                │
│ 📂 1) iris.csv  |  150 rows × 5 cols  |  3.77 KB │
╰──────────────────────────────────────────────────╯

                    🧾 Columns Overview                     
╭──────────────┬─────────┬────────────────┬────────────────╮
│ Column       │ Type    │ Non-Null Count │ Missing Values │
├──────────────┼─────────┼────────────────┼────────────────┤
│ sepal_length │ float64 │ 150            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ sepal_width  │ float64 │ 150            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ petal_length │ float64 │ 150            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ petal_width  │ float64 │ 150            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ species      │ object  │ 150            │ 0              │
╰──────────────┴─────────┴────────────────┴────────────────╯

╭───────────────────────────── Column Types ─────────────────────────────╮
│ 🔢 Numerical: 4 → sepal_length, sepal_width, petal_length, petal_width │
│ 🔤 Categorical: 1 → species                                            │
╰────────────────────────────────────────────────────────────────────────╯

╭─────────────────────╮
│ ✓ No Missing Values │
╰─────────────────────╯

╭──────────────────────╮
│ ✗ Duplicated Rows: 3 │
╰──────────────────────╯

/tmp/ipython-input-4018154006.py:55: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                                             📈 Descriptive Statistics                                             
╭──────────────┬───────┬────────┬────────┬──────┬────────────────┬─────────────────┬─────┬─────┬──────┬─────┬─────╮
│ Column       │ count │ unique │ top    │ freq │ mean           │ std             │ min │ 25% │ 50%  │ 75% │ max │
├──────────────┼───────┼────────┼────────┼──────┼────────────────┼─────────────────┼─────┼─────┼──────┼─────┼─────┤
│ sepal_length │ 150.0 │        │        │      │ 5.84333333333… │ 0.828066127977… │ 4.3 │ 5.1 │ 5.8  │ 6.4 │ 7.9 │
├──────────────┼───────┼────────┼────────┼──────┼────────────────┼─────────────────┼─────┼─────┼──────┼─────┼─────┤
│ sepal_width  │ 150.0 │        │        │      │ 3.05400000000… │ 0.433594311362… │ 2.0 │ 2.8 │ 3.0  │ 3.3 │ 4.4 │
├──────────────┼───────┼────────┼────────┼──────┼────────────────┼─────────────────┼─────┼─────┼──────┼─────┼─────┤
│ petal_length │ 150.0 │        │        │      │ 3.75866666666… │ 1.764420419952… │ 1.0 │ 1.6 │ 4.35 │ 5.1 │ 6.9 │
├──────────────┼───────┼────────┼────────┼──────┼────────────────┼─────────────────┼─────┼─────┼──────┼─────┼─────┤
│ petal_width  │ 150.0 │        │        │      │ 1.19866666666… │ 0.763160741700… │ 0.1 │ 0.3 │ 1.3  │ 1.8 │ 2.5 │
├──────────────┼───────┼────────┼────────┼──────┼────────────────┼─────────────────┼─────┼─────┼──────┼─────┼─────┤
│ species      │ 150.0 │ 3      │ setosa │ 50   │                │                 │     │     │      │     │     │
╰──────────────┴───────┴────────┴────────┴──────┴────────────────┴─────────────────┴─────┴─────┴──────┴─────┴─────╯

                           📝 First 5 Rows                           
╭──────────────┬─────────────┬──────────────┬─────────────┬─────────╮
│ sepal_length │ sepal_width │ petal_length │ petal_width │ species │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────┤
│ 5.1          │ 3.5         │ 1.4          │ 0.2         │ setosa  │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────┤
│ 4.9          │ 3.0         │ 1.4          │ 0.2         │ setosa  │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────┤
│ 4.7          │ 3.2         │ 1.3          │ 0.2         │ setosa  │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────┤
│ 4.6          │ 3.1         │ 1.5          │ 0.2         │ setosa  │
├──────────────┼─────────────┼──────────────┼─────────────┼─────────┤
│ 5.0          │ 3.6         │ 1.4          │ 0.2         │ setosa  │
╰──────────────┴─────────────┴──────────────┴─────────────┴─────────╯

In [461]:
dataset_report(df_house, file_path_house)

╭────────────────────────────────────────────────────────────────────────╮
│ 📊 Dataset Report                                                      │
│ 📂 4) house Prediction Data Set.csv  |  505 rows × 1 cols  |  47.93 KB │
╰────────────────────────────────────────────────────────────────────────╯

                                                🧾 Columns Overview                                                
╭──────────────────────────────────────────────────────────────────────┬────────┬────────────────┬────────────────╮
│ Column                                                               │ Type   │ Non-Null Count │ Missing Values │
├──────────────────────────────────────────────────────────────────────┼────────┼────────────────┼────────────────┤
│  0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0 │ object │ 505            │ 0              │
│ 15.30 396.90   4.98  24.00                                           │        │                │                │
╰──────────────────────────────────────────────────────────────────────┴────────┴────────────────┴────────────────╯

╭───────────────────────────────────────────────── Column Types ──────────────────────────────────────────────────╮
│ 🔢 Numerical: 0 →                                                                                               │
│ 🔤 Categorical: 1 →  0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98   │
│ 24.00                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────╮
│ ✓ No Missing Values │
╰─────────────────────╯

╭──────────────────────╮
│ ✓ No Duplicated Rows │
╰──────────────────────╯

/tmp/ipython-input-4018154006.py:55: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                                             📈 Descriptive Statistics                                             
╭────────────────────────────────────────────┬───────┬────────┬────────────────────────────────────────────┬──────╮
│ Column                                     │ count │ unique │ top                                        │ freq │
├────────────────────────────────────────────┼───────┼────────┼────────────────────────────────────────────┼──────┤
│  0.00632  18.00   2.310  0  0.5380  6.5750 │ 505   │ 505    │  0.04741   0.00  11.930  0  0.5730  6.0300 │ 1    │
│ 65.20  4.0900   1  296.0  15.30 396.90     │       │        │ 80.80  2.5050   1  273.0  21.00 396.90     │      │
│ 4.98  24.00                                │       │        │ 7.88  11.90                                │      │
╰────────────────────────────────────────────┴───────┴────────┴────────────────────────────────────────────┴──────╯

                                          📝 First 5 Rows                                           
╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│  0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98  24.00 │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│  0.02731   0.00   7.070  0  0.4690  6.4210  78.90  4.9671   2  242.0  17.80 396.90   9.14  21.60 │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│  0.02729   0.00   7.070  0  0.4690  7.1850  61.10  4.9671   2  242.0  17.80 392.83   4.03  34.70 │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│  0.03237   0.00   2.180  0  0.4580  6.9980  45.80  6.0622   3  222.0  18.70 394.63   2.94  33.40 │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│  0.06905   0.00   2.180  0  0.4580  7.1470  54.20  6.0622   3  222.0  18.70 396.90   5.33  36.20 │
├──────────────────────────────────────────────────────────────────────────────────────────────────┤
│  0.02985   0.00   2.180  0  0.4580  6.4300  58.70  6.0622   3  222.0  18.70 394.12   5.21  28.70 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [462]:
dataset_report(df_stock, file_path_stock)

╭──────────────────────────────────────────────────────────────────────────╮
│ 📊 Dataset Report                                                        │
│ 📂 2) Stock Prices Data Set.csv  |  497472 rows × 7 cols  |  23642.85 KB │
╰──────────────────────────────────────────────────────────────────────────╯

                 🧾 Columns Overview                  
╭────────┬─────────┬────────────────┬────────────────╮
│ Column │ Type    │ Non-Null Count │ Missing Values │
├────────┼─────────┼────────────────┼────────────────┤
│ symbol │ object  │ 497472         │ 0              │
├────────┼─────────┼────────────────┼────────────────┤
│ date   │ object  │ 497472         │ 0              │
├────────┼─────────┼────────────────┼────────────────┤
│ open   │ float64 │ 497461         │ 11             │
├────────┼─────────┼────────────────┼────────────────┤
│ high   │ float64 │ 497464         │ 8              │
├────────┼─────────┼────────────────┼────────────────┤
│ low    │ float64 │ 497464         │ 8              │
├────────┼─────────┼────────────────┼────────────────┤
│ close  │ float64 │ 497472         │ 0              │
├────────┼─────────┼────────────────┼────────────────┤
│ volume │ int64   │ 497472         │ 0              │
╰────────┴─────────┴────────────────┴────────────────╯

╭────────────────── Column Types ──────────────────╮
│ 🔢 Numerical: 5 → open, high, low, close, volume │
│ 🔤 Categorical: 2 → symbol, date                 │
╰──────────────────────────────────────────────────╯

     ⚠ Missing Values     
╭────────┬───────────────╮
│ Column │ Missing Count │
├────────┼───────────────┤
│ open   │ 11            │
├────────┼───────────────┤
│ high   │ 8             │
├────────┼───────────────┤
│ low    │ 8             │
╰────────┴───────────────╯

╭──────────────────────╮
│ ✓ No Duplicated Rows │
╰──────────────────────╯

/tmp/ipython-input-4018154006.py:55: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                                             📈 Descriptive Statistics                                             
╭────────┬─────────┬────────┬─────────┬──────┬─────────┬─────────┬──────┬──────────┬─────────┬──────────┬─────────╮
│ Column │ count   │ unique │ top     │ freq │ mean    │ std     │ min  │ 25%      │ 50%     │ 75%      │ max     │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ symbol │ 497472… │ 505    │ YUM     │ 1007 │         │         │      │          │         │          │         │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ date   │ 497472… │ 1007   │ 2017-1… │ 505  │         │         │      │          │         │          │         │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ open   │ 497461… │        │         │      │ 86.352… │ 101.47… │ 1.62 │ 41.69    │ 64.97   │ 98.41    │ 2044.0  │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ high   │ 497464… │        │         │      │ 87.132… │ 102.31… │ 1.69 │ 42.09    │ 65.56   │ 99.23    │ 2067.99 │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ low    │ 497464… │        │         │      │ 85.552… │ 100.57… │ 1.5  │ 41.28    │ 64.3537 │ 97.58    │ 2035.11 │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ close  │ 497472… │        │         │      │ 86.369… │ 101.47… │ 1.59 │ 41.70375 │ 64.98   │ 98.42    │ 2049.0  │
├────────┼─────────┼────────┼─────────┼──────┼─────────┼─────────┼──────┼──────────┼─────────┼──────────┼─────────┤
│ volume │ 497472… │        │         │      │ 425361… │ 823213… │ 0.0  │ 1080166… │ 208489… │ 4271928… │ 618237… │
╰────────┴─────────┴────────┴─────────┴──────┴─────────┴─────────┴──────┴──────────┴─────────┴──────────┴─────────╯

                             📝 First 5 Rows                              
╭────────┬────────────┬─────────┬─────────┬─────────┬─────────┬──────────╮
│ symbol │ date       │ open    │ high    │ low     │ close   │ volume   │
├────────┼────────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ AAL    │ 2014-01-02 │ 25.07   │ 25.82   │ 25.06   │ 25.36   │ 8998943  │
├────────┼────────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ AAPL   │ 2014-01-02 │ 79.3828 │ 79.5756 │ 78.8601 │ 79.0185 │ 58791957 │
├────────┼────────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ AAP    │ 2014-01-02 │ 110.36  │ 111.88  │ 109.29  │ 109.74  │ 542711   │
├────────┼────────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ ABBV   │ 2014-01-02 │ 52.12   │ 52.33   │ 51.52   │ 51.98   │ 4569061  │
├────────┼────────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│ ABC    │ 2014-01-02 │ 70.11   │ 70.23   │ 69.48   │ 69.89   │ 1148391  │
╰────────┴────────────┴─────────┴─────────┴─────────┴─────────┴──────────╯

In [463]:
dataset_report(df_sentiment, file_path_sentiment)

╭──────────────────────────────────────────────────────────────────╮
│ 📊 Dataset Report                                                │
│ 📂 3) Sentiment dataset.csv  |  732 rows × 15 cols  |  166.77 KB │
╰──────────────────────────────────────────────────────────────────╯

                    🧾 Columns Overview                     
╭──────────────┬─────────┬────────────────┬────────────────╮
│ Column       │ Type    │ Non-Null Count │ Missing Values │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Unnamed: 0.1 │ int64   │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Unnamed: 0   │ int64   │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Text         │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Sentiment    │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Timestamp    │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ User         │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Platform     │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Hashtags     │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Retweets     │ float64 │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Likes        │ float64 │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Country      │ object  │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Year         │ int64   │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Month        │ int64   │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Day          │ int64   │ 732            │ 0              │
├──────────────┼─────────┼────────────────┼────────────────┤
│ Hour         │ int64   │ 732            │ 0              │
╰──────────────┴─────────┴────────────────┴────────────────╯

╭─────────────────────────────────── Column Types ────────────────────────────────────╮
│ 🔢 Numerical: 8 → Unnamed: 0.1, Unnamed: 0, Retweets, Likes, Year, Month, Day, Hour │
│ 🔤 Categorical: 7 → Text, Sentiment, Timestamp, User, Platform, Hashtags, Country   │
╰─────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────╮
│ ✓ No Missing Values │
╰─────────────────────╯

╭──────────────────────╮
│ ✓ No Duplicated Rows │
╰──────────────────────╯

/tmp/ipython-input-4018154006.py:55: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                                             📈 Descriptive Statistics                                             
╭──────────┬───────┬────────┬──────────┬──────┬──────────┬───────────┬────────┬────────┬────────┬────────┬────────╮
│ Column   │ count │ unique │ top      │ freq │ mean     │ std       │ min    │ 25%    │ 50%    │ 75%    │ max    │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ Unnamed: │ 732.0 │        │          │      │ 366.464… │ 211.5139… │ 0.0    │ 183.75 │ 366.5  │ 549.25 │ 732.0  │
│ 0.1      │       │        │          │      │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ Unnamed: │ 732.0 │        │          │      │ 369.740… │ 212.4289… │ 0.0    │ 185.75 │ 370.5  │ 553.25 │ 736.0  │
│ 0        │       │        │          │      │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ Text     │ 732.0 │ 707    │  A       │ 3    │          │           │        │        │        │        │        │
│          │       │        │ compass… │      │          │           │        │        │        │        │        │
│          │       │        │ rain,    │      │          │           │        │        │        │        │        │
│          │       │        │ tears of │      │          │           │        │        │        │        │        │
│          │       │        │ empathy  │      │          │           │        │        │        │        │        │
│          │       │        │ falling  │      │          │           │        │        │        │        │        │
│          │       │        │ gently,  │      │          │           │        │        │        │        │        │
│          │       │        │ nurturi… │      │          │           │        │        │        │        │        │
│          │       │        │ the      │      │          │           │        │        │        │        │        │
│          │       │        │ seeds of │      │          │           │        │        │        │        │        │
│          │       │        │ kindness │      │          │           │        │        │        │        │        │
│          │       │        │ in the   │      │          │           │        │        │        │        │        │
│          │       │        │ garden   │      │          │           │        │        │        │        │        │
│          │       │        │ of human │      │          │           │        │        │        │        │        │
│          │       │        │ connect… │      │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ Sentime… │ 732.0 │ 279    │  Positi… │ 44   │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ Timesta… │ 732.0 │ 683    │ 2019-04… │ 3    │          │           │        │        │        │        │        │
│          │       │        │ 17:30:00 │      │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ User     │ 732.0 │ 685    │  Winter… │ 3    │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────┼───────────┼────────┼────────┼────────┼────────┼────────┤
│ Platform │ 732.0 │ 4      │  Instag… │ 258  │          │           │        │        │        │        │        │
├──────────┼───────┼────────┼──────────┼──────┼──────────

                                                  📝 First 5 Rows                                                  
╭───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬──────┬───────┬──────┬──────┬─────┬──────╮
│ Unnam │       │       │       │       │       │       │       │       │      │       │      │      │     │      │
│ ed:   │ Unnam │       │ Senti │ Times │       │ Platf │ Hasht │ Retwe │ Like │ Count │      │ Mont │     │      │
│ 0.1   │ ed: 0 │ Text  │ ment  │ tamp  │ User  │ orm   │ ags   │ ets   │ s    │ ry    │ Year │ h    │ Day │ Hour │
├───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼──────┼───────┼──────┼──────┼─────┼──────┤
│ 0     │ 0     │  Enjo │  Posi │ 2023- │  User │  Twit │  #Nat │ 15.0  │ 30.0 │  USA  │ 2023 │ 1    │ 15  │ 12   │
│       │       │ ying  │ tive  │ 01-15 │ 123   │ ter   │ ure   │       │      │       │      │      │     │      │
│       │       │ a     │       │       │       │       │ #Park │       │      │       │      │      │     │      │
│       │       │ beaut │       │ 12:30 │       │       │       │       │      │       │      │      │     │      │
│       │       │ iful  │       │ :00   │       │       │       │       │      │       │      │      │     │      │
│       │       │ day   │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │ at    │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │ the   │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │ park! │       │       │       │       │       │       │      │       │      │      │     │      │
├───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼──────┼───────┼──────┼──────┼─────┼──────┤
│ 1     │ 1     │  Traf │  Nega │ 2023- │  Comm │  Twit │  #Tra │ 5.0   │ 10.0 │  Cana │ 2023 │ 1    │ 15  │ 8    │
│       │       │ fic   │ tive  │ 01-15 │ uterX │ ter   │ ffic  │       │      │ da    │      │      │     │      │
│       │       │ was   │       │       │       │       │ #Morn │       │      │       │      │      │     │      │
│       │       │ terri │       │ 08:45 │       │       │ ing   │       │      │       │      │      │     │      │
│       │       │ ble   │       │ :00   │       │       │       │       │      │       │      │      │     │      │
│       │       │ this  │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │ morni │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │ ng.   │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │       │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │       │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │       │       │       │       │       │       │       │      │       │      │      │     │      │
├───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼──────┼───────┼──────┼──────┼─────┼──────┤
│ 2     │ 2     │  Just │  Posi │ 2023- │  Fitn │  Inst │  #Fit │ 20.0  │ 40.0 │  USA  │ 2023 │ 1    │ 15  │ 15   │
│       │       │ finis │ tive  │ 01-15 │ essFa │ agram │ ness  │       │      │       │      │      │     │      │
│       │       │ hed   │       │       │ n     │       │ #Work │       │      │       │      │      │     │      │
│       │       │ an    │       │ 15:45 │       │       │ out   │       │      │       │      │      │     │      │
│       │       │ amazi │       │ :00   │       │       │       │       │      │       │      │      │     │      │
│       │       │ ng    │       │       │       │       │       │       │      │       │      │      │     │      │
│       │       │ worko │       │       │       │       │

In [464]:
dataset_report(df_churn_20, file_path_churn_20)

╭───────────────────────────────────────────────────────────╮
│ 📊 Dataset Report                                         │
│ 📂 churn-bigml-20.csv  |  667 rows × 20 cols  |  55.01 KB │
╰───────────────────────────────────────────────────────────╯

                         🧾 Columns Overview                          
╭────────────────────────┬─────────┬────────────────┬────────────────╮
│ Column                 │ Type    │ Non-Null Count │ Missing Values │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ State                  │ object  │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Account length         │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Area code              │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ International plan     │ object  │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Voice mail plan        │ object  │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Number vmail messages  │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total day minutes      │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total day calls        │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total day charge       │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total eve minutes      │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total eve calls        │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total eve charge       │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total night minutes    │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total night calls      │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total night charge     │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total intl minutes     │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total intl calls       │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total intl charge      │ float64 │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Customer service calls │ int64   │ 667            │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Churn                  │ bool    │ 667            │ 0              │
╰────────────────────────┴─────────┴────────────────┴────────────────╯

╭───────────────────────────────────────────────── Column Types ──────────────────────────────────────────────────╮
│ 🔢 Numerical: 16 → Account length, Area code, Number vmail messages, Total day minutes, Total day calls, Total  │
│ day charge, Total eve minutes, Total eve calls, Total eve charge, Total night minutes ...                       │
│ 🔤 Categorical: 4 → State, International plan, Voice mail plan, Churn                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────╮
│ ✓ No Missing Values │
╰─────────────────────╯

╭──────────────────────╮
│ ✓ No Duplicated Rows │
╰──────────────────────╯

/tmp/ipython-input-4018154006.py:55: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                                             📈 Descriptive Statistics                                             
╭───────────┬───────┬────────┬───────┬──────┬───────────┬────────────┬───────┬───────────┬───────┬────────┬───────╮
│ Column    │ count │ unique │ top   │ freq │ mean      │ std        │ min   │ 25%       │ 50%   │ 75%    │ max   │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ State     │ 667.0 │ 51     │ AZ    │ 19   │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Account   │ 667.0 │        │       │      │ 102.8410… │ 40.819480… │ 1.0   │ 76.0      │ 102.0 │ 128.0  │ 232.0 │
│ length    │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Area code │ 667.0 │        │       │      │ 436.1574… │ 41.783304… │ 408.0 │ 408.0     │ 415.0 │ 415.0  │ 510.0 │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Internat… │ 667.0 │ 2      │ No    │ 614  │           │            │       │           │       │        │       │
│ plan      │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Voice     │ 667.0 │ 2      │ No    │ 478  │           │            │       │           │       │        │       │
│ mail plan │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Number    │ 667.0 │        │       │      │ 8.407796… │ 13.994479… │ 0.0   │ 0.0       │ 0.0   │ 20.0   │ 51.0  │
│ vmail     │       │        │       │      │           │            │       │           │       │        │       │
│ messages  │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Total day │ 667.0 │        │       │      │ 180.9481… │ 55.508627… │ 25.9  │ 146.25    │ 178.3 │ 220.7  │ 334.3 │
│ minutes   │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Total day │ 667.0 │        │       │      │ 100.9370… │ 20.396790… │ 30.0  │ 87.5      │ 101.0 │ 115.0  │ 165.0 │
│ calls     │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Total day │ 667.0 │        │       │      │ 30.76176… │ 9.4364634… │ 4.4   │ 24.86     │ 30.31 │ 37.52  │ 56.83 │
│ charge    │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Total eve │ 667.0 │        │       │      │ 203.3553… │ 49.719268… │ 48.1  │ 171.05    │ 203.7 │ 236.45 │ 361.8 │
│ minutes   │       │        │       │      │           │            │       │           │       │        │       │
├───────────┼───────┼────────┼───────┼──────┼───────────┼────────────┼───────┼───────────┼───────┼────────┼───────┤
│ Total eve │ 667.0 │        │       │      │ 100.4767… │ 18.948261… │ 37.0  │ 88.0      │ 101.0 │ 113.0  │ 168.0 │
│ calls     │       │        │       │      │           │

                                                  📝 First 5 Rows                                                  
╭─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬────┬─────┬────┬─────┬────┬─────┬────┬─────┬────┬─────┬────╮
│     │     │     │     │     │     │     │     │     │    │     │    │     │    │     │ To │     │    │     │    │
│     │     │     │     │     │     │     │     │     │ To │     │    │     │ To │     │ ta │     │ To │     │    │
│     │     │     │     │     │ Num │     │     │     │ ta │     │ To │     │ ta │     │ l  │     │ ta │ Cus │    │
│     │     │     │ Int │     │ ber │     │     │     │ l  │     │ ta │ Tot │ l  │     │ in │     │ l  │ tom │    │
│     │     │     │ ern │ Voi │     │ Tot │     │     │ ev │     │ l  │ al  │ ni │ Tot │ tl │ Tot │ in │ er  │    │
│     │ Acc │     │ ati │ ce  │ vma │ al  │ Tot │ Tot │ e  │ Tot │ ev │ nig │ gh │ al  │    │ al  │ tl │ ser │    │
│     │ oun │ Are │ ona │ mai │ il  │ day │ al  │ al  │ mi │ al  │ e  │ ht  │ t  │ nig │ mi │ int │    │ vic │    │
│     │ t   │ a   │ l   │ l   │ mes │ min │ day │ day │ nu │ eve │ ch │ min │ ca │ ht  │ nu │ l   │ ch │ e   │ Ch │
│ Sta │ len │ cod │ pla │ pla │ sag │ ute │ cal │ cha │ te │ cal │ ar │ ute │ ll │ cha │ te │ cal │ ar │ cal │ ur │
│ te  │ gth │ e   │ n   │ n   │ es  │ s   │ ls  │ rge │ s  │ ls  │ ge │ s   │ s  │ rge │ s  │ ls  │ ge │ ls  │ n  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ LA  │ 117 │ 408 │ No  │ No  │ 0   │ 184 │ 97  │ 31. │ 35 │ 80  │ 29 │ 215 │ 90 │ 9.7 │ 8. │ 4   │ 2. │ 1   │ Fa │
│     │     │     │     │     │     │ .5  │     │ 37  │ 1. │     │ .8 │ .8  │    │ 1   │ 7  │     │ 35 │     │ ls │
│     │     │     │     │     │     │     │     │     │ 6  │     │ 9  │     │    │     │    │     │    │     │ e  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ IN  │ 65  │ 415 │ No  │ No  │ 0   │ 129 │ 137 │ 21. │ 22 │ 83  │ 19 │ 208 │ 11 │ 9.4 │ 12 │ 6   │ 3. │ 4   │ Tr │
│     │     │     │     │     │     │ .1  │     │ 95  │ 8. │     │ .4 │ .8  │ 1  │     │ .7 │     │ 43 │     │ ue │
│     │     │     │     │     │     │     │     │     │ 5  │     │ 2  │     │    │     │    │     │    │     │    │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ NY  │ 161 │ 415 │ No  │ No  │ 0   │ 332 │ 67  │ 56. │ 31 │ 97  │ 27 │ 160 │ 12 │ 7.2 │ 5. │ 9   │ 1. │ 4   │ Tr │
│     │     │     │     │     │     │ .9  │     │ 59  │ 7. │     │ .0 │ .6  │ 8  │ 3   │ 4  │     │ 46 │     │ ue │
│     │     │     │     │     │     │     │     │     │ 8  │     │ 1  │     │    │     │    │     │    │     │    │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ SC  │ 111 │ 415 │ No  │ No  │ 0   │ 110 │ 103 │ 18. │ 13 │ 102 │ 11 │ 189 │ 10 │ 8.5 │ 7. │ 6   │ 2. │ 2   │ Fa │
│     │     │     │     │     │     │ .4  │     │ 77  │ 7. │     │ .6 │ .6  │ 5  │ 3   │ 7  │     │ 08 │     │ ls │
│     │     │     │     │     │     │     │     │     │ 3  │     │ 7  │     │    │     │    │     │    │     │ e  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ HI  │ 49  │ 510 │ No  │ No  │ 0   │ 119 │ 117 │ 20. │ 21 │ 109 │ 18 │ 178 │ 90 │ 8.0 │ 11 │ 1   │ 3. │ 1   │ Fa │
│     │     │     │     │     │     │ .3  │     │ 28  │ 5. │     │ .2 │ .7  │    │ 4   │ .1 │     │ 0  │     │ ls │
│     │     │     │     │     │     │     │     │     │ 1  │     │ 8  │     │    │     │    │     │    │     │ e  │
╰─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴────┴─────┴────┴─────┴────┴─────┴────┴─────┴────┴─────┴────╯

In [465]:
dataset_report(df_churn_80, file_path_churn_80)

╭─────────────────────────────────────────────────────────────╮
│ 📊 Dataset Report                                           │
│ 📂 churn-bigml-80.csv  |  2666 rows × 20 cols  |  218.75 KB │
╰─────────────────────────────────────────────────────────────╯

                         🧾 Columns Overview                          
╭────────────────────────┬─────────┬────────────────┬────────────────╮
│ Column                 │ Type    │ Non-Null Count │ Missing Values │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ State                  │ object  │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Account length         │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Area code              │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ International plan     │ object  │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Voice mail plan        │ object  │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Number vmail messages  │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total day minutes      │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total day calls        │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total day charge       │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total eve minutes      │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total eve calls        │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total eve charge       │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total night minutes    │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total night calls      │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total night charge     │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total intl minutes     │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total intl calls       │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Total intl charge      │ float64 │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Customer service calls │ int64   │ 2666           │ 0              │
├────────────────────────┼─────────┼────────────────┼────────────────┤
│ Churn                  │ bool    │ 2666           │ 0              │
╰────────────────────────┴─────────┴────────────────┴────────────────╯

╭───────────────────────────────────────────────── Column Types ──────────────────────────────────────────────────╮
│ 🔢 Numerical: 16 → Account length, Area code, Number vmail messages, Total day minutes, Total day calls, Total  │
│ day charge, Total eve minutes, Total eve calls, Total eve charge, Total night minutes ...                       │
│ 🔤 Categorical: 4 → State, International plan, Voice mail plan, Churn                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────╮
│ ✓ No Missing Values │
╰─────────────────────╯

╭──────────────────────╮
│ ✓ No Duplicated Rows │
╰──────────────────────╯

/tmp/ipython-input-4018154006.py:55: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



                                             📈 Descriptive Statistics                                             
╭──────────┬────────┬────────┬───────┬──────┬──────────┬──────────┬───────┬──────────┬──────────┬─────────┬───────╮
│ Column   │ count  │ unique │ top   │ freq │ mean     │ std      │ min   │ 25%      │ 50%      │ 75%     │ max   │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ State    │ 2666.0 │ 51     │ WV    │ 88   │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Account  │ 2666.0 │        │       │      │ 100.620… │ 39.5639… │ 1.0   │ 73.0     │ 100.0    │ 127.0   │ 243.0 │
│ length   │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Area     │ 2666.0 │        │       │      │ 437.438… │ 42.5210… │ 408.0 │ 408.0    │ 415.0    │ 510.0   │ 510.0 │
│ code     │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Interna… │ 2666.0 │ 2      │ No    │ 2396 │          │          │       │          │          │         │       │
│ plan     │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Voice    │ 2666.0 │ 2      │ No    │ 1933 │          │          │       │          │          │         │       │
│ mail     │        │        │       │      │          │          │       │          │          │         │       │
│ plan     │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Number   │ 2666.0 │        │       │      │ 8.02175… │ 13.6122… │ 0.0   │ 0.0      │ 0.0      │ 19.0    │ 50.0  │
│ vmail    │        │        │       │      │          │          │       │          │          │         │       │
│ messages │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Total    │ 2666.0 │        │       │      │ 179.481… │ 54.2103… │ 0.0   │ 143.4    │ 179.95   │ 215.9   │ 350.8 │
│ day      │        │        │       │      │          │          │       │          │          │         │       │
│ minutes  │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Total    │ 2666.0 │        │       │      │ 100.310… │ 19.9881… │ 0.0   │ 87.0     │ 101.0    │ 114.0   │ 160.0 │
│ day      │        │        │       │      │          │          │       │          │          │         │       │
│ calls    │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼──────────┼───────┼──────────┼──────────┼─────────┼───────┤
│ Total    │ 2666.0 │        │       │      │ 30.5124… │ 9.21573… │ 0.0   │ 24.38    │ 30.59    │ 36.7    │ 59.64 │
│ day      │        │        │       │      │          │          │       │          │          │         │       │
│ charge   │        │        │       │      │          │          │       │          │          │         │       │
├──────────┼────────┼────────┼───────┼──────┼──────────┼─

                                                  📝 First 5 Rows                                                  
╭─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬────┬─────┬────┬─────┬────┬─────┬────┬─────┬────┬─────┬────╮
│     │     │     │     │     │     │     │     │     │    │     │    │     │    │     │ To │     │    │     │    │
│     │     │     │     │     │     │     │     │     │ To │     │    │     │ To │     │ ta │     │ To │     │    │
│     │     │     │     │     │ Num │     │     │     │ ta │     │ To │     │ ta │     │ l  │     │ ta │ Cus │    │
│     │     │     │ Int │     │ ber │     │     │     │ l  │     │ ta │ Tot │ l  │     │ in │     │ l  │ tom │    │
│     │     │     │ ern │ Voi │     │ Tot │     │     │ ev │     │ l  │ al  │ ni │ Tot │ tl │ Tot │ in │ er  │    │
│     │ Acc │     │ ati │ ce  │ vma │ al  │ Tot │ Tot │ e  │ Tot │ ev │ nig │ gh │ al  │    │ al  │ tl │ ser │    │
│     │ oun │ Are │ ona │ mai │ il  │ day │ al  │ al  │ mi │ al  │ e  │ ht  │ t  │ nig │ mi │ int │    │ vic │    │
│     │ t   │ a   │ l   │ l   │ mes │ min │ day │ day │ nu │ eve │ ch │ min │ ca │ ht  │ nu │ l   │ ch │ e   │ Ch │
│ Sta │ len │ cod │ pla │ pla │ sag │ ute │ cal │ cha │ te │ cal │ ar │ ute │ ll │ cha │ te │ cal │ ar │ cal │ ur │
│ te  │ gth │ e   │ n   │ n   │ es  │ s   │ ls  │ rge │ s  │ ls  │ ge │ s   │ s  │ rge │ s  │ ls  │ ge │ ls  │ n  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ KS  │ 128 │ 415 │ No  │ Yes │ 25  │ 265 │ 110 │ 45. │ 19 │ 99  │ 16 │ 244 │ 91 │ 11. │ 10 │ 3   │ 2. │ 1   │ Fa │
│     │     │     │     │     │     │ .1  │     │ 07  │ 7. │     │ .7 │ .7  │    │ 01  │ .0 │     │ 7  │     │ ls │
│     │     │     │     │     │     │     │     │     │ 4  │     │ 8  │     │    │     │    │     │    │     │ e  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ OH  │ 107 │ 415 │ No  │ Yes │ 26  │ 161 │ 123 │ 27. │ 19 │ 103 │ 16 │ 254 │ 10 │ 11. │ 13 │ 3   │ 3. │ 1   │ Fa │
│     │     │     │     │     │     │ .6  │     │ 47  │ 5. │     │ .6 │ .4  │ 3  │ 45  │ .7 │     │ 7  │     │ ls │
│     │     │     │     │     │     │     │     │     │ 5  │     │ 2  │     │    │     │    │     │    │     │ e  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ NJ  │ 137 │ 415 │ No  │ No  │ 0   │ 243 │ 114 │ 41. │ 12 │ 110 │ 10 │ 162 │ 10 │ 7.3 │ 12 │ 5   │ 3. │ 0   │ Fa │
│     │     │     │     │     │     │ .4  │     │ 38  │ 1. │     │ .3 │ .6  │ 4  │ 2   │ .2 │     │ 29 │     │ ls │
│     │     │     │     │     │     │     │     │     │ 2  │     │    │     │    │     │    │     │    │     │ e  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ OH  │ 84  │ 408 │ Yes │ No  │ 0   │ 299 │ 71  │ 50. │ 61 │ 88  │ 5. │ 196 │ 89 │ 8.8 │ 6. │ 7   │ 1. │ 2   │ Fa │
│     │     │     │     │     │     │ .4  │     │ 9   │ .9 │     │ 26 │ .9  │    │ 6   │ 6  │     │ 78 │     │ ls │
│     │     │     │     │     │     │     │     │     │    │     │    │     │    │     │    │     │    │     │ e  │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┼─────┼────┤
│ OK  │ 75  │ 415 │ Yes │ No  │ 0   │ 166 │ 113 │ 28. │ 14 │ 122 │ 12 │ 186 │ 12 │ 8.4 │ 10 │ 3   │ 2. │ 3   │ Fa │
│     │     │     │     │     │     │ .7  │     │ 34  │ 8. │     │ .6 │ .9  │ 1  │ 1   │ .1 │     │ 73 │     │ ls │
│     │     │     │     │     │     │     │     │     │ 3  │     │ 1  │     │    │     │    │     │    │     │ e  │
╰─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴────┴─────┴────┴─────┴────┴─────┴────┴─────┴────┴─────┴────╯

## **1.3 Dataset Observations & Insights**

---
**🌸 Iris Dataset (1)**
* The dataset contains `150 rows` × `5 columns` with `4 numerical` features (sepal & petal dimensions) and `1 categorical target` (species).
* **No missing values**, but **3 duplicated rows exist**, which may need removal before model training.
* All features are cleanly separated into numeric and categorical, making it ideal for classification tasks.
* Well-structured and balanced dataset — a great candidate for supervised ML experiments.

---
**🏠 House Prediction Dataset (4)**
* The dataset shows `505 rows` × `1 column`, but the entire row is stored as a single object column.
* This indicates the dataset is not properly delimited (possibly missing commas or separators in the CSV).
* Currently, no missing values or duplicates, but parsing is required to split the data into meaningful columns (e.g., price, rooms, location, etc.).
* Needs preprocessing before it can be used in ML task

---
**📈 Stock Prices Dataset (2)**
* Extremely large dataset with `497,472 rows` × `7 columns` (symbols, date, OHLC prices, and volume).
* Contains `11 missing` values in open, 8 in `high`, and 8 in `low` → very small relative to dataset size but should be imputed or dropped.
* No duplicate rows detected.
* A mix of categorical (`symbol`, `date`) and numerical (`prices`, `volume`) data makes it suitable for time-series forecasting & financial analysis.

---
**💬 Sentiment Dataset (3)**
* `732 rows` × `15 columns`, covering text, sentiment, user metadata, and engagement metrics (likes, retweets).
* **No missing values** and **no duplicates**, dataset is clean.
* Features include a mix of textual data (`Text`, `Hashtags`), categorical metadata (`User`, `Platform`, `Country`), and numerical fields (`Retweets`, `Likes`, `Year`, etc.).
* Well-suited for **NLP sentiment classification** with potential for feature engineering from metadata.
---
**📞 Churn Dataset (Churn-bigml-80)**
* `2666 rows` × `20 columns`, covering customer demographics, service usage, and churn label.
* Balanced data types: 16 numerical columns (call minutes, charges, service usage) and 4 categorical columns (state, plans, churn).
* **No missing or duplicate values** — dataset is well-structured and ready for ML.
* Directly applicable for classification tasks (Churn prediction) and can be used for building explainable ML models.
---
**📞 Churn Dataset (Churn-bigml-20)**
* Contains `667 rows` × `20 columns`, making it a relatively small dataset, ideal for exploratory analysis and model prototyping.
* Numerical Features (16): Includes usage metrics such as total day minutes, total eve calls, total intl charge, customer service calls, etc.
* Categorical Features (4): State, International plan, Voice mail plan, and the target variable → Churn.
* No Missing Values: Ensures data completeness.
* No Duplicated Rows: Data is clean and unique across entries.

## **1.4 Preprocess Datasets**

In [466]:
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Checks and removes duplicate rows in a DataFrame.

    Args:
        df (pd.DataFrame): Input dataset

    Returns:
        pd.DataFrame: DataFrame with duplicates removed
    """
    # Before
    rows_before = len(df)
    duplicates_before = df.duplicated().sum()

    # Drop duplicates
    df_cleaned = df.drop_duplicates()

    # After
    rows_after = len(df_cleaned)
    duplicates_after = df_cleaned.duplicated().sum()

    # Pretty summary table
    table = Table(title="🔍 Duplicate Check Summary", show_lines=True)
    table.add_column("Metric", style="cyan", justify="left")
    table.add_column("Before", style="yellow", justify="center")
    table.add_column("After", style="green", justify="center")

    table.add_row("Rows", str(rows_before), str(rows_after))
    table.add_row("Duplicate Rows", str(duplicates_before), str(duplicates_after))

    console.print(table)

    return df_cleaned

In [467]:
def handle_missing_data(df, strategy="mean"):
    console.rule("[bold yellow]Handling Missing Data")

    missing_before = df.isnull().sum().sum()
    console.print(f"[cyan]Total Missing Values Before: [bold red]{missing_before}")

    if strategy == "mean":
        df = df.fillna(df.mean(numeric_only=True))
    elif strategy == "median":
        df = df.fillna(df.median(numeric_only=True))
    elif strategy == "drop":
        df = df.dropna()
    else:
        console.print("[red]Invalid strategy. Choose 'mean', 'median', or 'drop'.")
        return df

    missing_after = df.isnull().sum().sum()
    console.print(f"[cyan]Total Missing Values After: [bold green]{missing_after}")
    return df


In [468]:
def encode_categorical(df, method="onehot", sample_size=5):
    """
    Encode categorical variables in a DataFrame using One-Hot or Label Encoding.

    Parameters:
        df (pd.DataFrame): Input DataFrame
        method (str): Encoding method ("onehot" or "label")
        sample_size (int): Number of rows to show in before/after preview

    Returns:
        pd.DataFrame: Encoded DataFrame
    """
    console.rule("[bold yellow]🔤 Encoding Categorical Variables")

    # Identify categorical columns
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    if not cat_cols:
        console.print("[red]No categorical columns found.[/red]")
        return df

    # Print summary before encoding
    console.print(f"[cyan]Categorical Columns Detected:[/cyan] {cat_cols}")
    console.print(f"[magenta]Shape Before:[/magenta] {df.shape[0]} rows × {df.shape[1]} columns")

    # Show a sample before encoding
    table_before = Table(title="📊 Sample Data (Before Encoding)", show_lines=True)
    for col in df.columns:
        table_before.add_column(col, style="cyan", overflow="fold")
    for _, row in df.head(sample_size).iterrows():
        table_before.add_row(*[str(x) for x in row.values])
    console.print(table_before)

    # Apply encoding
    if method == "onehot":
        df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    elif method == "label":
        df_encoded = df.copy()
        le = LabelEncoder()
        for col in cat_cols:
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    else:
        console.print("[red]❌ Invalid method. Choose 'onehot' or 'label'.")
        return df

    # Print summary after encoding
    console.print(f"[green]✅ Encoding done using: [bold]{method.upper()}[/bold]")
    console.print(f"[magenta]Shape After:[/magenta] {df_encoded.shape[0]} rows × {df_encoded.shape[1]} columns")

    # Show a sample after encoding
    table_after = Table(title="📊 Sample Data (After Encoding)", show_lines=True)
    for col in df_encoded.columns:
        table_after.add_column(col, style="green", overflow="fold")
    for _, row in df_encoded.head(sample_size).iterrows():
        table_after.add_row(*[str(x) for x in row.values])
    console.print(table_after)

    return df_encoded

In [469]:
def split_data(df, target, test_size=0.2, val_size=0.1):
    console.rule("[bold yellow]Splitting Dataset")

    X = df.drop(columns=[target])
    y = df[target]

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size+val_size, random_state=42)
    relative_val_size = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=relative_val_size, random_state=42)

    console.print(f"[cyan]Train Size: {X_train.shape[0]}")
    console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
    console.print(f"[cyan]Test Size: {X_test.shape[0]}")

    return X_train, X_val, X_test, y_train, y_val, y_test

In [470]:
def split_dataset(X, y, test_size=0.2, val_size=0.1, random_state=42):
    """
    Splits dataset into train, validation, and test sets with stratification.

    Args:
        X (pd.DataFrame): Features
        y (pd.Series): Target labels
        test_size (float): Proportion for test set
        val_size (float): Proportion for validation set (from remaining after test)
        random_state (int): Random state for reproducibility

    Returns:
        dict: Dictionary containing X_train, X_val, X_test, y_train, y_val, y_test
    """

    console.rule("[bold green] Data Splitting")

    # First split into Train+Val and Test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # Adjust val_size relative to remaining set
    val_adjusted = val_size / (1 - test_size)

    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_adjusted, stratify=y_temp, random_state=random_state
    )

    # Print dataset summary in rich table
    table = Table(title="Dataset Split Summary", show_header=True, header_style="bold magenta")
    table.add_column("Set", justify="center")
    table.add_column("Shape (X)", justify="center")
    table.add_column("Shape (y)", justify="center")
    table.add_column("Class Distribution", justify="center")

    for name, X_part, y_part in [
        ("Train", X_train, y_train),
        ("Validation", X_val, y_val),
        ("Test", X_test, y_test)
    ]:
        class_dist = y_part.value_counts().to_dict()
        table.add_row(name, str(X_part.shape), str(y_part.shape), str(class_dist))

    console.print(table)

    # Show sample rows for each split
    console.rule("[bold yellow]Sample Rows")
    for name, X_part, y_part in [
        ("Train", X_train, y_train),
        ("Validation", X_val, y_val),
        ("Test", X_test, y_test)
    ]:
        console.print(f"[cyan]{name} Sample[/cyan]:")
        console.print(pd.concat([X_part.head(3), y_part.head(3)], axis=1))
        console.print("\n")

    return {
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "y_train": y_train, "y_val": y_val, "y_test": y_test
    }

In [471]:
def standardize_numeric(X_train, X_val, X_test, scaler_path="models/scaler.pkl"):
    console.rule("[bold yellow]Standardizing Numerical Features")

    num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    console.print(f"[cyan]Numerical Columns: {num_cols}")

    console.print("[red]Before Standardization (Train Set):")
    console.print(X_train[num_cols].agg(['mean', 'std']))

    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_val[num_cols] = scaler.transform(X_val[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])

    console.print("[green]After Standardization (Train Set):")
    console.print(X_train[num_cols].agg(['mean', 'std']))
    # Save scaler
    os.makedirs(os.path.dirname(scaler_path), exist_ok=True)
    joblib.dump((scaler, num_cols), scaler_path)
    console.print(f"[green]✅ Scaler saved at {scaler_path}[/green]")


    return X_train, X_val, X_test

In [472]:
def save_preprocessed(data: dict, filename: str):
    """
    Save preprocessed dataset dictionary to a pickle file.

    Parameters
    ----------
    data : dict
        Dictionary containing train/val/test splits.
    filename : str
        Name of the file to save (e.g., 'preprocessed.pkl').
    """
    filepath = Path(filename)
    with open(filepath, "wb") as f:
        pickle.dump(data, f)
    print(f"✅ Preprocessed dataset saved to {filepath.resolve()}")


In [473]:
def load_preprocessed(filename: str) -> dict:
    """
    Load preprocessed dataset dictionary from a pickle file.

    Parameters
    ----------
    filename : str
        Path to the saved pickle file.

    Returns
    -------
    dict
        Dictionary containing train/val/test splits.
    """
    filepath = Path(filename)
    if not filepath.exists():
        raise FileNotFoundError(f"❌ File not found: {filepath.resolve()}")

    with open(filepath, "rb") as f:
        data = pickle.load(f)

    print(f"✅ Preprocessed dataset loaded from {filepath.resolve()}")
    return data

### **1.4.1 Preprocess Iris dataset**

In [474]:
# Step 1: Remove Duplicates
df_clean = remove_duplicates(df_iris)

    🔍 Duplicate Check Summary     
┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Metric         ┃ Before ┃ After ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ Rows           │  150   │  147  │
├────────────────┼────────┼───────┤
│ Duplicate Rows │   3    │   0   │
└────────────────┴────────┴───────┘

In [475]:
# Print number of rows
console.print(f"[cyan]Total Rows: {df_clean.shape[0]}")

Total Rows: 147

In [476]:
# Step 2: Handle missing
df_clean = handle_missing_data(df_clean, strategy="median")

────────────────────────────────────────────── Handling Missing Data ──────────────────────────────────────────────

Total Missing Values Before: 0

Total Missing Values After: 0

In [477]:
# Separate features (X) and target (y)
X = df_clean.drop("species", axis=1)
y = df_clean["species"]

In [478]:
result = split_dataset(X, y, test_size=0.2, val_size=0.1, random_state=42)

─────────────────────────────────────────────────  Data Splitting ─────────────────────────────────────────────────

                                  Dataset Split Summary                                   
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃    Set     ┃ Shape (X) ┃ Shape (y) ┃                Class Distribution                 ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   Train    │ (102, 4)  │  (102,)   │ {'versicolor': 35, 'virginica': 34, 'setosa': 33} │
│ Validation │  (15, 4)  │   (15,)   │  {'virginica': 5, 'versicolor': 5, 'setosa': 5}   │
│    Test    │  (30, 4)  │   (30,)   │ {'versicolor': 10, 'virginica': 10, 'setosa': 10} │
└────────────┴───────────┴───────────┴───────────────────────────────────────────────────┘

─────────────────────────────────────────────────── Sample Rows ───────────────────────────────────────────────────

Train Sample:

sepal_length  sepal_width  petal_length  petal_width    species
38            4.4          3.0           1.3          0.2     setosa
8             4.4          2.9           1.4          0.2     setosa
110           6.5          3.2           5.1          2.0  virginica

Validation Sample:

sepal_length  sepal_width  petal_length  petal_width     species
101           5.8          2.7           5.1          1.9   virginica
81            5.5          2.4           3.7          1.0  versicolor
17            5.1          3.5           1.4          0.3      setosa

Test Sample:

sepal_length  sepal_width  petal_length  petal_width     species
63            6.1          2.9           4.7          1.4  versicolor
84            5.4          3.0           4.5          1.5  versicolor
141           6.9          3.1           5.1          2.3   virginica

In [479]:
X_train_std, X_val_std, X_test_std = standardize_numeric(
    result["X_train"].copy(),
    result["X_val"].copy(),
    result["X_test"].copy()
)

──────────────────────────────────────── Standardizing Numerical Features ─────────────────────────────────────────

Numerical Columns: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

Before Standardization (Train Set):

sepal_length  sepal_width  petal_length  petal_width
mean      5.848039     3.017647      3.810784     1.200000
std       0.885497     0.434049      1.777188     0.751632

After Standardization (Train Set):

sepal_length   sepal_width  petal_length   petal_width
mean  1.431317e-15  1.306145e-16  1.319750e-16  1.050358e-16
std   1.004938e+00  1.004938e+00  1.004938e+00  1.004938e+00

✅ Scaler saved at models/scaler.pkl

In [480]:
result["X_train"] = X_train_std
result["X_val"] = X_val_std
result["X_test"] = X_test_std

In [481]:
save_preprocessed(result, "data/preprocessed/preprocessed_iris.pkl")
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_iris.pkl")

✅ Preprocessed dataset saved to /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_iris.pkl
✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_iris.pkl


In [377]:
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test = loaded_splits["X_test"], loaded_splits["y_test"]

# Print train/validate and test to make sure that we loaded them sucessfully
console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

Train Size: 102

Validation Size: 15

Test Size: 30

### **1.4.2 Preprocess Stock dataset**

In [378]:
# Step 1: Remove Duplicates
df_clean = remove_duplicates(df_stock)

     🔍 Duplicate Check Summary     
┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ Metric         ┃ Before ┃ After  ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ Rows           │ 497472 │ 497472 │
├────────────────┼────────┼────────┤
│ Duplicate Rows │   0    │   0    │
└────────────────┴────────┴────────┘

In [379]:
# Step 2: Handle missing
df_clean = handle_missing_data(df_clean, strategy="median")

────────────────────────────────────────────── Handling Missing Data ──────────────────────────────────────────────

Total Missing Values Before: 27

Total Missing Values After: 0

In [380]:
# drop symbol column
df_clean = df_clean.drop("symbol", axis=1)

In [381]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df_clean, target = "close")

──────────────────────────────────────────────── Splitting Dataset ────────────────────────────────────────────────

Train Size: 348230

Validation Size: 99494

Test Size: 49748

In [382]:
X_train_std, X_val_std, X_test_std = standardize_numeric(
    X_train.copy(),
    X_val.copy(),
    X_test.copy()
)

──────────────────────────────────────── Standardizing Numerical Features ─────────────────────────────────────────

Numerical Columns: ['open', 'high', 'low', 'volume']

Before Standardization (Train Set):

open        high         low        volume
mean   86.360609   87.140512   85.560368  4.230722e+06
std   101.261818  102.102183  100.372079  8.109386e+06

After Standardization (Train Set):

open          high           low        volume
mean  7.325183e-17  2.986389e-16  5.556121e-17 -1.305882e-18
std   1.000001e+00  1.000001e+00  1.000001e+00  1.000001e+00

✅ Scaler saved at models/scaler.pkl

In [383]:
# Convert splits into dictionary
preprocessed_data = {
    "X_train": X_train_std,
    "X_val": X_val_std,
    "X_test": X_test_std,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [384]:
save_preprocessed(preprocessed_data, "data/preprocessed/preprocessed_stock.pkl")
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_stock.pkl")

✅ Preprocessed dataset saved to /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_stock.pkl
✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_stock.pkl


In [385]:
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test = loaded_splits["X_test"], loaded_splits["y_test"]

# Print train/validate and test to make sure that we loaded them sucessfully
console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

Train Size: 348230

Validation Size: 99494

Test Size: 49748

### **1.4.3 Preprocess house dataset**

In [386]:
# Load properly (handles spaces/tabs as separators)
df_house = pd.read_csv(file_path_house, delim_whitespace=True, header=None)

print(df_house.shape)
print(df_house.head())

(506, 14)
        0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12    13  
0  396.90  4.98  24.0  
1  396.90  9.14  21.6  
2  392.83  4.03  34.7  
3  394.63  2.94  33.4  
4  396.90  5.33  36.2  


/tmp/ipython-input-3208306138.py:2: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead



In [387]:
# Assume we will use hte columns dataset from kaggle dataset
df_house.columns = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS",
    "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

1.  `CRIM` – Per capita crime rate by town.
2.  `ZN` – Proportion of residential land zoned for lots over 25,000 sq. ft.
3.  `INDUS` – Proportion of non-retail business acres per town.
4.  `CHAS` – Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5.  `NOX` – Nitric oxides concentration (parts per 10 million).
6.  `RM` – Average number of rooms per dwelling.
7.  `AGE` – Proportion of owner-occupied units built prior to 1940.
8.  `DIS` – Weighted distances to five Boston employment centres.
9.  `RAD` – Index of accessibility to radial highways.
10. `TAX` – Full-value property-tax rate per $10,000.
11. `PTRATIO` – Pupil-teacher ratio by town.
12. `B` – 1000(Bk − 0.63)² where Bk is the proportion of Black residents by town.
13. `LSTAT` – % lower status of the population.

In [388]:
# Step 1: Remove Duplicates
df_clean = remove_duplicates(df_house)

    🔍 Duplicate Check Summary     
┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Metric         ┃ Before ┃ After ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ Rows           │  506   │  506  │
├────────────────┼────────┼───────┤
│ Duplicate Rows │   0    │   0   │
└────────────────┴────────┴───────┘

In [389]:
# Step 2: Handle missing
df_clean = handle_missing_data(df_clean, strategy="median")

────────────────────────────────────────────── Handling Missing Data ──────────────────────────────────────────────

Total Missing Values Before: 0

Total Missing Values After: 0

In [390]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df_clean, target = "MEDV")

──────────────────────────────────────────────── Splitting Dataset ────────────────────────────────────────────────

Train Size: 354

Validation Size: 101

Test Size: 51

In [391]:
X_train_std, X_val_std, X_test_std = standardize_numeric(
    X_train.copy(),
    X_val.copy(),
    X_test.copy()
)

──────────────────────────────────────── Standardizing Numerical Features ─────────────────────────────────────────

Numerical Columns: ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 
'LSTAT']

Before Standardization (Train Set):

CRIM         ZN      INDUS      CHAS       NOX        RM        AGE  \
mean  3.469887  11.403955  11.133051  0.073446  0.557259  6.325672  68.799718   
std   8.315831  22.608457   6.938651  0.261237  0.116792  0.719211  27.665360   

           DIS       RAD         TAX    PTRATIO           B      LSTAT  
mean  3.765874  9.437853  407.042373  18.277966  359.701808  12.421130  
std   2.126032  8.639971  166.522237   2.256792   86.924779   7.112402

After Standardization (Train Set):

CRIM            ZN         INDUS          CHAS           NOX  \
mean  0.000000 -5.017957e-18  3.763468e-16  1.003591e-17 -6.874601e-16   
std   1.001415  1.001415e+00  1.001415e+00  1.001415e+00  1.001415e+00   

                RM           AGE           DIS           RAD           TAX  \
mean -3.512570e-16 -6.774242e-17 -1.405028e-16 -6.272446e-17 -1.756285e-16   
std   1.001415e+00  1.001415e+00  1.001415e+00  1.001415e+00  1.001415e+00   

           PTRATIO             B         LSTAT  
mean  4.014366e-17 -4.541251e-16 -2.508979e-18  
std   1.001415e+00  1.001415e+00  1.001415e+00

✅ Scaler saved at models/scaler.pkl

In [392]:
result["X_train"] = X_train_std
result["X_val"] = X_val_std
result["X_test"] = X_test_std

In [393]:
# Convert splits into dictionary
preprocessed_data = {
    "X_train": X_train_std,
    "X_val": X_val_std,
    "X_test": X_test_std,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [394]:
save_preprocessed(preprocessed_data, "data/preprocessed/preprocessed_house.pkl")
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_house.pkl")

✅ Preprocessed dataset saved to /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_house.pkl
✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_house.pkl


In [395]:
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test = loaded_splits["X_test"], loaded_splits["y_test"]

# Print train/validate and test to make sure that we loaded them sucessfully
console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

Train Size: 354

Validation Size: 101

Test Size: 51

### **1.4.4 Preprocess churn-bigml-20 dataset**

In [396]:
# Step 1: Remove Duplicates
df_clean = remove_duplicates(df_churn_20)

    🔍 Duplicate Check Summary     
┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Metric         ┃ Before ┃ After ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ Rows           │  667   │  667  │
├────────────────┼────────┼───────┤
│ Duplicate Rows │   0    │   0   │
└────────────────┴────────┴───────┘

In [397]:
# Step 2: Handle missing
df_clean = handle_missing_data(df_clean, strategy="median")

────────────────────────────────────────────── Handling Missing Data ──────────────────────────────────────────────

Total Missing Values Before: 0

Total Missing Values After: 0

In [398]:
# Print cols names
df_clean.columns

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')

In [399]:
# Print cols types
df_clean.dtypes

,0
State,object
Account length,int64
Area code,int64
International plan,object
Voice mail plan,object
Number vmail messages,int64
Total day minutes,float64
Total day calls,int64
Total day charge,float64
Total eve minutes,float64


In [400]:
df_clean.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False


In [401]:
# Find unique values for objects
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        unique_values = df_clean[col].unique()
        print(f"Unique values for column '{col}':")
        print(unique_values)

Unique values for column 'State':
['LA' 'IN' 'NY' 'SC' 'HI' 'AK' 'MI' 'ID' 'VA' 'WI' 'MN' 'VT' 'MT' 'MA'
 'KY' 'CO' 'AZ' 'CA' 'WA' 'NE' 'OH' 'MO' 'AL' 'NH' 'NM' 'OR' 'TX' 'MS'
 'WY' 'FL' 'KS' 'NC' 'SD' 'OK' 'CT' 'RI' 'DE' 'UT' 'NV' 'DC' 'ME' 'IL'
 'NJ' 'MD' 'WV' 'PA' 'ND' 'AR' 'TN' 'IA' 'GA']
Unique values for column 'International plan':
['No' 'Yes']
Unique values for column 'Voice mail plan':
['No' 'Yes']


In [402]:
# # Define features and target
X = df_clean.drop(columns=["Churn"])   # Features (everything except target)
y = df_clean["Churn"]                  # Target

In [403]:
df_encoded = encode_categorical(X, method="onehot")

──────────────────────────────────────── 🔤 Encoding Categorical Variables ────────────────────────────────────────

Categorical Columns Detected: ['State', 'International plan', 'Voice mail plan']

Shape Before: 667 rows × 19 columns

                                         📊 Sample Data (Before Encoding)                                          
┏━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃     ┃     ┃     ┃     ┃     ┃ Num ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃ Cus ┃
┃     ┃     ┃     ┃ Int ┃     ┃ ber ┃     ┃     ┃     ┃     ┃     ┃     ┃ Tot ┃     ┃     ┃ Tot ┃     ┃     ┃ tom ┃
┃     ┃     ┃     ┃ ern ┃ Voi ┃     ┃ Tot ┃     ┃     ┃ Tot ┃     ┃     ┃ al  ┃ Tot ┃ Tot ┃ al  ┃ Tot ┃ Tot ┃ er  ┃
┃     ┃ Acc ┃     ┃ ati ┃ ce  ┃ vma ┃ al  ┃ Tot ┃ Tot ┃ al  ┃ Tot ┃ Tot ┃ nig ┃ al  ┃ al  ┃ int ┃ al  ┃ al  ┃ ser ┃
┃     ┃ oun ┃ Are ┃ ona ┃ mai ┃ il  ┃ day ┃ al  ┃ al  ┃ eve ┃ al  ┃ al  ┃ ht  ┃ nig ┃ nig ┃ l   ┃ int ┃ int ┃ vic ┃
┃     ┃ t   ┃ a   ┃ l   ┃ l   ┃ mes ┃ min ┃ day ┃ day ┃ min ┃ eve ┃ eve ┃ min ┃ ht  ┃ ht  ┃ min ┃ l   ┃ l   ┃ e   ┃
┃ Sta ┃ len ┃ cod ┃ pla ┃ pla ┃ sag ┃ ute ┃ cal ┃ cha ┃ ute ┃ cal ┃ cha ┃ ute ┃ cal ┃ cha ┃ ute ┃ cal ┃ cha ┃ cal ┃
┃ te  ┃ gth ┃ e   ┃ n   ┃ n   ┃ es  ┃ s   ┃ ls  ┃ rge ┃ s   ┃ ls  ┃ rge ┃ s   ┃ ls  ┃ rge ┃ s   ┃ ls  ┃ rge ┃ ls  ┃
┡━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│ LA  │ 117 │ 408 │ No  │ No  │ 0   │ 184 │ 97  │ 31. │ 351 │ 80  │ 29. │ 215 │ 90  │ 9.7 │ 8.7 │ 4   │ 2.3 │ 1   │
│     │     │     │     │     │     │ .5  │     │ 37  │ .6  │     │ 89  │ .8  │     │ 1   │     │     │ 5   │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ IN  │ 65  │ 415 │ No  │ No  │ 0   │ 129 │ 137 │ 21. │ 228 │ 83  │ 19. │ 208 │ 111 │ 9.4 │ 12. │ 6   │ 3.4 │ 4   │
│     │     │     │     │     │     │ .1  │     │ 95  │ .5  │     │ 42  │ .8  │     │     │ 7   │     │ 3   │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ NY  │ 161 │ 415 │ No  │ No  │ 0   │ 332 │ 67  │ 56. │ 317 │ 97  │ 27. │ 160 │ 128 │ 7.2 │ 5.4 │ 9   │ 1.4 │ 4   │
│     │     │     │     │     │     │ .9  │     │ 59  │ .8  │     │ 01  │ .6  │     │ 3   │     │     │ 6   │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ SC  │ 111 │ 415 │ No  │ No  │ 0   │ 110 │ 103 │ 18. │ 137 │ 102 │ 11. │ 189 │ 105 │ 8.5 │ 7.7 │ 6   │ 2.0 │ 2   │
│     │     │     │     │     │     │ .4  │     │ 77  │ .3  │     │ 67  │ .6  │     │ 3   │     │     │ 8   │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ HI  │ 49  │ 510 │ No  │ No  │ 0   │ 119 │ 117 │ 20. │ 215 │ 109 │ 18. │ 178 │ 90  │ 8.0 │ 11. │ 1   │ 3.0 │ 1   │
│     │     │     │     │     │     │ .3  │     │ 28  │ .1  │     │ 28  │ .7  │     │ 4   │ 1   │     │     │     │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘

✅ Encoding done using: ONEHOT

Shape After: 667 rows × 68 columns

                                          📊 Sample Data (After Encoding)                                          
┏━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┓
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃
┡━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━┩
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
└─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┘

In [404]:
# combine X and y into df_encoded to split them
df_encoded["Churn"] = y

In [405]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(
      df_encoded, target="Churn"
  )

──────────────────────────────────────────────── Splitting Dataset ────────────────────────────────────────────────

Train Size: 466

Validation Size: 134

Test Size: 67

In [406]:
X_train_std, X_val_std, X_test_std = standardize_numeric(
    X_train.copy(),
    X_val.copy(),
    X_test.copy()
)

──────────────────────────────────────── Standardizing Numerical Features ─────────────────────────────────────────

Numerical Columns: ['Account length', 'Area code', 'Number vmail messages', 'Total day minutes', 'Total day calls',
'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night
calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service 
calls']

Before Standardization (Train Set):

Account length   Area code  Number vmail messages  Total day minutes  \
mean      102.839056  438.000000               8.590129         180.034764   
std        40.576940   42.778499              14.130172          55.394347   

      Total day calls  Total day charge  Total eve minutes  Total eve calls  \
mean       101.650215         30.606438         204.099785       100.081545   
std         20.987553          9.417181          49.456957        19.083456   

      Total eve charge  Total night minutes  Total night calls  \
mean         17.348476           196.672318         100.633047   
std           4.203822            49.285408          20.104900   

      Total night charge  Total intl minutes  Total intl calls  \
mean            8.850408           10.223605          4.472103   
std             2.218120            2.819784          2.424843   

      Total intl charge  Customer service calls  
mean           2.761030                1.538627  
std            0.761442                1.316402

After Standardization (Train Set):

Account length     Area code  Number vmail messages  Total day minutes  \
mean   -8.767426e-17  2.668347e-17           5.717887e-18      -2.534930e-16   
std     1.001075e+00  1.001075e+00           1.001075e+00       1.001075e+00   

      Total day calls  Total day charge  Total eve minutes  Total eve calls  \
mean    -3.049540e-17     -3.621328e-17      -2.715996e-16     1.986966e-16   
std      1.001075e+00      1.001075e+00       1.001075e+00     1.001075e+00   

      Total eve charge  Total night minutes  Total night calls  \
mean      6.089549e-16         4.660078e-16      -1.734426e-16   
std       1.001075e+00         1.001075e+00       1.001075e+00   

      Total night charge  Total intl minutes  Total intl calls  \
mean       -1.715366e-17       -5.012681e-16     -7.623849e-17   
std         1.001075e+00        1.001075e+00      1.001075e+00   

      Total intl charge  Customer service calls  
mean      -5.317635e-16            4.193117e-17  
std        1.001075e+00            1.001075e+00

✅ Scaler saved at models/scaler.pkl

In [407]:
result["X_train"] = X_train_std
result["X_val"] = X_val_std
result["X_test"] = X_test_std

In [408]:
# Convert splits into dictionary
preprocessed_data = {
    "X_train": X_train_std,
    "X_val": X_val_std,
    "X_test": X_test_std,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [409]:
save_preprocessed(preprocessed_data, "data/preprocessed/preprocessed_churn_20.pkl")
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_churn_20.pkl")

✅ Preprocessed dataset saved to /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_churn_20.pkl
✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_churn_20.pkl


In [410]:
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test = loaded_splits["X_test"], loaded_splits["y_test"]

# Print train/validate and test to make sure that we loaded them sucessfully
console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

Train Size: 466

Validation Size: 134

Test Size: 67

### **1.4.5 Preprocess churn-bigml-80 dataset**

In [411]:
# Step 1: Remove Duplicates
df_clean = remove_duplicates(df_churn_80)

    🔍 Duplicate Check Summary     
┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Metric         ┃ Before ┃ After ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ Rows           │  2666  │ 2666  │
├────────────────┼────────┼───────┤
│ Duplicate Rows │   0    │   0   │
└────────────────┴────────┴───────┘

In [412]:
# Step 2: Handle missing
df_clean = handle_missing_data(df_clean, strategy="median")

────────────────────────────────────────────── Handling Missing Data ──────────────────────────────────────────────

Total Missing Values Before: 0

Total Missing Values After: 0

In [413]:
# Print cols names
df_clean.columns

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')

In [414]:
# Print cols types
df_clean.dtypes

,0
State,object
Account length,int64
Area code,int64
International plan,object
Voice mail plan,object
Number vmail messages,int64
Total day minutes,float64
Total day calls,int64
Total day charge,float64
Total eve minutes,float64


In [415]:
df_clean.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [416]:
# Find unique values for objects
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        unique_values = df_clean[col].unique()
        print(f"Unique values for column '{col}':")
        print(unique_values)

Unique values for column 'State':
['KS' 'OH' 'NJ' 'OK' 'AL' 'MA' 'MO' 'WV' 'RI' 'IA' 'MT' 'ID' 'VT' 'VA'
 'TX' 'FL' 'CO' 'AZ' 'NE' 'WY' 'IL' 'NH' 'LA' 'GA' 'AK' 'MD' 'AR' 'WI'
 'OR' 'DE' 'IN' 'UT' 'CA' 'SD' 'NC' 'WA' 'MN' 'NM' 'NV' 'DC' 'NY' 'KY'
 'ME' 'MS' 'MI' 'SC' 'TN' 'PA' 'HI' 'ND' 'CT']
Unique values for column 'International plan':
['No' 'Yes']
Unique values for column 'Voice mail plan':
['Yes' 'No']


In [417]:
# # Define features and target
X = df_clean.drop(columns=["Churn"])   # Features (everything except target)
y = df_clean["Churn"]                  # Target

In [418]:
df_encoded = encode_categorical(X, method="onehot")

──────────────────────────────────────── 🔤 Encoding Categorical Variables ────────────────────────────────────────

Categorical Columns Detected: ['State', 'International plan', 'Voice mail plan']

Shape Before: 2666 rows × 19 columns

                                         📊 Sample Data (Before Encoding)                                          
┏━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃     ┃     ┃     ┃     ┃     ┃ Num ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃     ┃ Cus ┃
┃     ┃     ┃     ┃ Int ┃     ┃ ber ┃     ┃     ┃     ┃     ┃     ┃     ┃ Tot ┃     ┃     ┃ Tot ┃     ┃     ┃ tom ┃
┃     ┃     ┃     ┃ ern ┃ Voi ┃     ┃ Tot ┃     ┃     ┃ Tot ┃     ┃     ┃ al  ┃ Tot ┃ Tot ┃ al  ┃ Tot ┃ Tot ┃ er  ┃
┃     ┃ Acc ┃     ┃ ati ┃ ce  ┃ vma ┃ al  ┃ Tot ┃ Tot ┃ al  ┃ Tot ┃ Tot ┃ nig ┃ al  ┃ al  ┃ int ┃ al  ┃ al  ┃ ser ┃
┃     ┃ oun ┃ Are ┃ ona ┃ mai ┃ il  ┃ day ┃ al  ┃ al  ┃ eve ┃ al  ┃ al  ┃ ht  ┃ nig ┃ nig ┃ l   ┃ int ┃ int ┃ vic ┃
┃     ┃ t   ┃ a   ┃ l   ┃ l   ┃ mes ┃ min ┃ day ┃ day ┃ min ┃ eve ┃ eve ┃ min ┃ ht  ┃ ht  ┃ min ┃ l   ┃ l   ┃ e   ┃
┃ Sta ┃ len ┃ cod ┃ pla ┃ pla ┃ sag ┃ ute ┃ cal ┃ cha ┃ ute ┃ cal ┃ cha ┃ ute ┃ cal ┃ cha ┃ ute ┃ cal ┃ cha ┃ cal ┃
┃ te  ┃ gth ┃ e   ┃ n   ┃ n   ┃ es  ┃ s   ┃ ls  ┃ rge ┃ s   ┃ ls  ┃ rge ┃ s   ┃ ls  ┃ rge ┃ s   ┃ ls  ┃ rge ┃ ls  ┃
┡━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│ KS  │ 128 │ 415 │ No  │ Yes │ 25  │ 265 │ 110 │ 45. │ 197 │ 99  │ 16. │ 244 │ 91  │ 11. │ 10. │ 3   │ 2.7 │ 1   │
│     │     │     │     │     │     │ .1  │     │ 07  │ .4  │     │ 78  │ .7  │     │ 01  │ 0   │     │     │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ OH  │ 107 │ 415 │ No  │ Yes │ 26  │ 161 │ 123 │ 27. │ 195 │ 103 │ 16. │ 254 │ 103 │ 11. │ 13. │ 3   │ 3.7 │ 1   │
│     │     │     │     │     │     │ .6  │     │ 47  │ .5  │     │ 62  │ .4  │     │ 45  │ 7   │     │     │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ NJ  │ 137 │ 415 │ No  │ No  │ 0   │ 243 │ 114 │ 41. │ 121 │ 110 │ 10. │ 162 │ 104 │ 7.3 │ 12. │ 5   │ 3.2 │ 0   │
│     │     │     │     │     │     │ .4  │     │ 38  │ .2  │     │ 3   │ .6  │     │ 2   │ 2   │     │ 9   │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ OH  │ 84  │ 408 │ Yes │ No  │ 0   │ 299 │ 71  │ 50. │ 61. │ 88  │ 5.2 │ 196 │ 89  │ 8.8 │ 6.6 │ 7   │ 1.7 │ 2   │
│     │     │     │     │     │     │ .4  │     │ 9   │ 9   │     │ 6   │ .9  │     │ 6   │     │     │ 8   │     │
├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ OK  │ 75  │ 415 │ Yes │ No  │ 0   │ 166 │ 113 │ 28. │ 148 │ 122 │ 12. │ 186 │ 121 │ 8.4 │ 10. │ 3   │ 2.7 │ 3   │
│     │     │     │     │     │     │ .7  │     │ 34  │ .3  │     │ 61  │ .9  │     │ 1   │ 1   │     │ 3   │     │
└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘

✅ Encoding done using: ONEHOT

Shape After: 2666 rows × 68 columns

                                          📊 Sample Data (After Encoding)                                          
┏━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┳┳━┓
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃┃ ┃
┡━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━╇╇━┩
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
├─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┼┼─┤
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ ││ │
└─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┴┴─┘

In [419]:
# combine X and y into df_encoded to split them
df_encoded["Churn"] = y

In [420]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(
      df_encoded, target="Churn"
  )

──────────────────────────────────────────────── Splitting Dataset ────────────────────────────────────────────────

Train Size: 1866

Validation Size: 533

Test Size: 267

In [421]:
X_train_std, X_val_std, X_test_std = standardize_numeric(
    X_train.copy(),
    X_val.copy(),
    X_test.copy()
)

──────────────────────────────────────── Standardizing Numerical Features ─────────────────────────────────────────

Numerical Columns: ['Account length', 'Area code', 'Number vmail messages', 'Total day minutes', 'Total day calls',
'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night
calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service 
calls']

Before Standardization (Train Set):

Account length   Area code  Number vmail messages  Total day minutes  \
mean      101.489282  437.689175               7.894962         180.788532   
std        39.730825   42.660954              13.539524          53.426715   

      Total day calls  Total day charge  Total eve minutes  Total eve calls  \
mean       100.367631         30.734582         200.059271       100.046088   
std         19.769446          9.082529          51.285880        20.162344   

      Total eve charge  Total night minutes  Total night calls  \
mean         17.005252           201.794641         100.233655   
std           4.359302            51.191744          19.306504   

      Total night charge  Total intl minutes  Total intl calls  \
mean            9.080884           10.182583          4.496249   
std             2.303562            2.743988          2.505166   

      Total intl charge  Customer service calls  
mean           2.749748                1.549839  
std            0.740817                1.310683

After Standardization (Train Set):

Account length     Area code  Number vmail messages  Total day minutes  \
mean   -1.028116e-16 -5.825993e-16           3.046271e-17      -3.179545e-16   
std     1.000268e+00  1.000268e+00           1.000268e+00       1.000268e+00   

      Total day calls  Total day charge  Total eve minutes  Total eve calls  \
mean     1.389861e-16     -1.446979e-16       3.788800e-16     1.123312e-16   
std      1.000268e+00      1.000268e+00       1.000268e+00     1.000268e+00   

      Total eve charge  Total night minutes  Total night calls  \
mean      2.627409e-16         1.142352e-17      -1.142352e-16   
std       1.000268e+00         1.000268e+00       1.000268e+00   

      Total night charge  Total intl minutes  Total intl calls  \
mean       -3.998231e-17       -5.330974e-17      1.865841e-16   
std         1.000268e+00        1.000268e+00      1.000268e+00   

      Total intl charge  Customer service calls  
mean      -2.370380e-16           -5.711758e-17  
std        1.000268e+00            1.000268e+00

✅ Scaler saved at models/scaler.pkl

In [422]:
result["X_train"] = X_train_std
result["X_val"] = X_val_std
result["X_test"] = X_test_std

In [423]:
# Convert splits into dictionary
preprocessed_data = {
    "X_train": X_train_std,
    "X_val": X_val_std,
    "X_test": X_test_std,
    "y_train": y_train,
    "y_val": y_val,
    "y_test": y_test
}

In [424]:
save_preprocessed(preprocessed_data, "data/preprocessed/preprocessed_churn_80.pkl")
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_churn_80.pkl")

✅ Preprocessed dataset saved to /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_churn_80.pkl
✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_churn_80.pkl


In [425]:
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test = loaded_splits["X_test"], loaded_splits["y_test"]

# Print train/validate and test to make sure that we loaded them sucessfully
console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

Train Size: 1866

Validation Size: 533

Test Size: 267

# **Task Two : Build a Simple Linear Regression Model**

  * **Description**: Build a linear regression model to predicta continuous variable (e.g., house prices).

  
**Objective**:
  * Load a dataset and preprocess it.
  * Train a linear regression model using scikit-learn.
  * Interpret he model coefficients
  * Evaluate the model using R-Squared and mean squared error (MSE)
  * Tools: Python, Pandas, scikit-learn.



In [426]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from rich.console import Console
from rich.table import Table
from rich.box import ROUNDED
import plotly.express as px
import plotly.graph_objects as go
import joblib

## **2.1 Load Preprocessed Splits**

In [427]:
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_house.pkl")
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val     = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test   = loaded_splits["X_test"], loaded_splits["y_test"]

console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_house.pkl


Train Size: 354

Validation Size: 101

Test Size: 51

## **2.2 Define the models**

In [428]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.01),
    "Decision Tree": DecisionTreeRegressor(max_depth=6, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42)
}

## **2.3 Train & Evaluate Models**

In [429]:
def evaluate_regression_model(name, model, X_train, y_train, X_val, y_val, X_test, y_test):
    results = {"Model": name}

    # Fit model
    model.fit(X_train, y_train)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_val_pred   = model.predict(X_val)
    y_test_pred  = model.predict(X_test)

    # Train metrics
    results["Train R²"]   = r2_score(y_train, y_train_pred)
    results["Train MSE"]  = mean_squared_error(y_train, y_train_pred)
    results["Train RMSE"] = np.sqrt(results["Train MSE"])
    results["Train MAE"]  = mean_absolute_error(y_train, y_train_pred)

    # Validation metrics
    results["Val R²"]   = r2_score(y_val, y_val_pred)
    results["Val MSE"]  = mean_squared_error(y_val, y_val_pred)
    results["Val RMSE"] = np.sqrt(results["Val MSE"])
    results["Val MAE"]  = mean_absolute_error(y_val, y_val_pred)

    # Test metrics
    results["Test R²"]   = r2_score(y_test, y_test_pred)
    results["Test MSE"]  = mean_squared_error(y_test, y_test_pred)
    results["Test RMSE"] = np.sqrt(results["Test MSE"])
    results["Test MAE"]  = mean_absolute_error(y_test, y_test_pred)

    # Pretty print with Rich
    table = Table(title=f"{name} — Performance", show_lines=True, box=ROUNDED)
    table.add_column("Dataset", style="bold cyan")
    table.add_column("R²", style="bold green")
    table.add_column("MSE", style="bold yellow")
    table.add_column("RMSE", style="bold magenta")
    table.add_column("MAE", style="bold red")

    for split in ["Train", "Val", "Test"]:
        table.add_row(
            split,
            f"{results[f'{split} R²']:.4f}",
            f"{results[f'{split} MSE']:.2f}",
            f"{results[f'{split} RMSE']:.2f}",
            f"{results[f'{split} MAE']:.2f}"
        )

    console.print(table)

    return results

In [430]:
def visualize_results(results_list, metric="Val R²"):
    """
    results_list: list of dicts from evaluate_regression_model
    metric: str → which metric to visualize (e.g., 'Val R²', 'Test RMSE')
    """
    df = pd.DataFrame(results_list)
    fig = px.bar(df, x="Model", y=metric, text=metric,
                 title=f"Model Comparison: {metric}",
                 color="Model")
    fig.update_traces(texttemplate="%{text:.3f}", textposition="outside")
    fig.update_layout(title_x=0.5, uniformtext_minsize=8, uniformtext_mode="hide")
    fig.show()

In [431]:
all_results = []
for name, model in models.items():
    res = evaluate_regression_model(name, model, X_train, y_train, X_val, y_val, X_test, y_test)
    all_results.append(res)

     Linear Regression — Performance      
╭─────────┬────────┬───────┬──────┬──────╮
│ Dataset │ R²     │ MSE   │ RMSE │ MAE  │
├─────────┼────────┼───────┼──────┼──────┤
│ Train   │ 0.7435 │ 22.55 │ 4.75 │ 3.36 │
├─────────┼────────┼───────┼──────┼──────┤
│ Val     │ 0.7079 │ 17.54 │ 4.19 │ 3.05 │
├─────────┼────────┼───────┼──────┼──────┤
│ Test    │ 0.7140 │ 29.39 │ 5.42 │ 3.39 │
╰─────────┴────────┴───────┴──────┴──────╯

      Ridge Regression — Performance      
╭─────────┬────────┬───────┬──────┬──────╮
│ Dataset │ R²     │ MSE   │ RMSE │ MAE  │
├─────────┼────────┼───────┼──────┼──────┤
│ Train   │ 0.7435 │ 22.55 │ 4.75 │ 3.35 │
├─────────┼────────┼───────┼──────┼──────┤
│ Val     │ 0.7076 │ 17.56 │ 4.19 │ 3.05 │
├─────────┼────────┼───────┼──────┼──────┤
│ Test    │ 0.7135 │ 29.45 │ 5.43 │ 3.39 │
╰─────────┴────────┴───────┴──────┴──────╯

      Lasso Regression — Performance      
╭─────────┬────────┬───────┬──────┬──────╮
│ Dataset │ R²     │ MSE   │ RMSE │ MAE  │
├─────────┼────────┼───────┼──────┼──────┤
│ Train   │ 0.7434 │ 22.55 │ 4.75 │ 3.35 │
├─────────┼────────┼───────┼──────┼──────┤
│ Val     │ 0.7073 │ 17.58 │ 4.19 │ 3.05 │
├─────────┼────────┼───────┼──────┼──────┤
│ Test    │ 0.7133 │ 29.47 │ 5.43 │ 3.38 │
╰─────────┴────────┴───────┴──────┴──────╯

       Decision Tree — Performance        
╭─────────┬────────┬───────┬──────┬──────╮
│ Dataset │ R²     │ MSE   │ RMSE │ MAE  │
├─────────┼────────┼───────┼──────┼──────┤
│ Train   │ 0.9479 │ 4.58  │ 2.14 │ 1.57 │
├─────────┼────────┼───────┼──────┼──────┤
│ Val     │ 0.8547 │ 8.72  │ 2.95 │ 2.18 │
├─────────┼────────┼───────┼──────┼──────┤
│ Test    │ 0.8665 │ 13.72 │ 3.70 │ 2.76 │
╰─────────┴────────┴───────┴──────┴──────╯

       Random Forest — Performance        
╭─────────┬────────┬───────┬──────┬──────╮
│ Dataset │ R²     │ MSE   │ RMSE │ MAE  │
├─────────┼────────┼───────┼──────┼──────┤
│ Train   │ 0.9764 │ 2.07  │ 1.44 │ 0.92 │
├─────────┼────────┼───────┼──────┼──────┤
│ Val     │ 0.8866 │ 6.81  │ 2.61 │ 1.87 │
├─────────┼────────┼───────┼──────┼──────┤
│ Test    │ 0.8522 │ 15.19 │ 3.90 │ 2.52 │
╰─────────┴────────┴───────┴──────┴──────╯

In [432]:
visualize_results(all_results, metric="Val R²")
visualize_results(all_results, metric="Test RMSE")

In [433]:
def analyze_best_model(results_list, models_dict,
                       X_train, y_train, X_val, y_val, X_test, y_test,
                       criterion="Val R²"):
    """
    Selects the best model based on criterion (default Val R²),
    prints coefficients/feature importances,
    and plots predicted vs actual + residuals.
    """
    # Convert results to DataFrame
    df = pd.DataFrame(results_list)
    best_row = df.sort_values(by=criterion, ascending=False).iloc[0]
    best_model_name = best_row["Model"]
    best_model = models_dict[best_model_name]

    console.print(f"[bold green]🏆 Best Model Detected: {best_model_name}[/bold green]")

    # Retrain best model on Train + Val for full capacity
    X_full = pd.concat([X_train, X_val])
    y_full = pd.concat([y_train, y_val])
    best_model.fit(X_full, y_full)

    # Predict on Test
    y_pred_test = best_model.predict(X_test)

    # ===============================
    # 1) Coefficients / Feature Importance
    # ===============================
    if hasattr(best_model, "coef_"):
        coefs = best_model.coef_
        features = X_train.columns if hasattr(X_train, "columns") else [f"f{i}" for i in range(len(coefs))]

        coef_table = Table(title=f"{best_model_name} — Top Coefficients", show_lines=True, box=ROUNDED)
        coef_table.add_column("Feature", style="bold cyan")
        coef_table.add_column("Coefficient", style="bold yellow")

        top_idx = np.argsort(np.abs(coefs))[::-1][:15]  # top 15
        for i in top_idx:
            coef_table.add_row(features[i], f"{coefs[i]:.4f}")
        console.print(coef_table)

    elif hasattr(best_model, "feature_importances_"):
        imps = best_model.feature_importances_
        features = X_train.columns if hasattr(X_train, "columns") else [f"f{i}" for i in range(len(imps))]

        imp_table = Table(title=f"{best_model_name} — Feature Importances", show_lines=True, box=ROUNDED)
        imp_table.add_column("Feature", style="bold cyan")
        imp_table.add_column("Importance", style="bold yellow")

        top_idx = np.argsort(imps)[::-1][:15]
        for i in top_idx:
            imp_table.add_row(features[i], f"{imps[i]:.4f}")
        console.print(imp_table)

    else:
        console.print("[yellow]⚠ No coefficients or feature importances available for this model[/yellow]")

    # ===============================
    # 2) Predicted vs Actual Plot
    # ===============================
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=y_test, y=y_pred_test, mode="markers",
                              name="Predicted vs Actual",
                              marker=dict(color="cyan", size=7, opacity=0.7)))
    fig1.add_trace(go.Scatter(x=y_test, y=y_test, mode="lines",
                              name="Ideal Fit", line=dict(color="red", dash="dash")))

    fig1.update_layout(
        title=f"{best_model_name} — Predicted vs Actual (Test Set)",
        title_x=0.5,
        xaxis_title="Actual Values",
        yaxis_title="Predicted Values",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(0,0,0,0)")
    )
    fig1.show()

    # ===============================
    # 3) Residual Plot
    # ===============================
    residuals = y_test - y_pred_test
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=y_pred_test, y=residuals, mode="markers",
                              name="Residuals", marker=dict(color="orange", size=7, opacity=0.7)))
    fig2.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Zero Line")

    fig2.update_layout(
        title=f"{best_model_name} — Residuals vs Predicted (Test Set)",
        title_x=0.5,
        xaxis_title="Predicted Values",
        yaxis_title="Residuals",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(0,0,0,0)")
    )
    fig2.show()
    return best_model

In [434]:
best_model = analyze_best_model(all_results, models,
                   X_train, y_train, X_val, y_val, X_test, y_test,
                   criterion="Val R²")

🏆 Best Model Detected: Random Forest

Random Forest — Feature 
      Importances       
╭─────────┬────────────╮
│ Feature │ Importance │
├─────────┼────────────┤
│ RM      │ 0.4837     │
├─────────┼────────────┤
│ LSTAT   │ 0.3399     │
├─────────┼────────────┤
│ DIS     │ 0.0579     │
├─────────┼────────────┤
│ CRIM    │ 0.0263     │
├─────────┼────────────┤
│ NOX     │ 0.0246     │
├─────────┼────────────┤
│ PTRATIO │ 0.0177     │
├─────────┼────────────┤
│ AGE     │ 0.0143     │
├─────────┼────────────┤
│ TAX     │ 0.0116     │
├─────────┼────────────┤
│ B       │ 0.0107     │
├─────────┼────────────┤
│ INDUS   │ 0.0064     │
├─────────┼────────────┤
│ RAD     │ 0.0037     │
├─────────┼────────────┤
│ CHAS    │ 0.0018     │
├─────────┼────────────┤
│ ZN      │ 0.0014     │
╰─────────┴────────────╯

In [435]:
def save_model(model, filepath: str):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    joblib.dump(model, filepath)
    console.print(f"[green]✅ Model saved at:[/green] {filepath}")

In [436]:
def load_model(filepath: str):
    if not os.path.exists(filepath):
        console.print(f"[red]❌ Model not found at:[/red] {filepath}")
        return None
    model = joblib.load(filepath)
    console.print(f"[green]✅ Model loaded from:[/green] {filepath}")
    return model

In [437]:
save_model(best_model, "models/house_price_best_model.pkl")

✅ Model saved at: models/house_price_best_model.pkl

In [438]:
loaded_model = load_model("models/house_price_best_model.pkl")

✅ Model loaded from: models/house_price_best_model.pkl

In [439]:
y_pred_loaded = loaded_model.predict(X_test)
console.print(f"[cyan]Prediction sample:[/cyan] {y_pred_loaded[:5]}")

Prediction sample: [28.058 13.153 37.747 20.036 21.272]

# Level One :

**Task Three : Implement K-Nearest Neighbors (KNN) Classifier**

  * **Description**: Build a KNN to classify data points into categories.

  
**Objective**:
  * Train a KNN model on a labeled dataset.
  * Evaluate the performance using accuracy, confusion matrix and precision/recall.
  * Use different values of K and compare the results.
  * Evaluate the model using R-Squared and mean squared error (MSE)
  * Tools: Python, Pandas, scikit-learn.



In [440]:
!pip install optuna

In [441]:
from sklearn.neighbors import KNeighborsClassifier
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, r2_score, mean_squared_error
import plotly.figure_factory as ff

In [442]:
# load preprocessed dataset
loaded_splits = load_preprocessed("data/preprocessed/preprocessed_iris.pkl")
X_train, y_train = loaded_splits["X_train"], loaded_splits["y_train"]
X_val, y_val     = loaded_splits["X_val"], loaded_splits["y_val"]
X_test, y_test   = loaded_splits["X_test"], loaded_splits["y_test"]

console.print(f"[cyan]Train Size: {X_train.shape[0]}")
console.print(f"[cyan]Validation Size: {X_val.shape[0]}")
console.print(f"[cyan]Test Size: {X_test.shape[0]}")

✅ Preprocessed dataset loaded from /content/drive/MyDrive/Codveda Technologies/data/preprocessed/preprocessed_iris.pkl


Train Size: 102

Validation Size: 15

Test Size: 30

In [443]:
def objective_knn(trial, X_train, y_train, X_val, y_val):
    k = trial.suggest_int("n_neighbors", 3, 20)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

def objective_logreg(trial, X_train, y_train, X_val, y_val):
    C = trial.suggest_float("C", 0.01, 10.0, log=True)
    solver = trial.suggest_categorical("solver", ["liblinear", "lbfgs"])
    model = LogisticRegression(C=C, solver=solver, max_iter=200)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

def objective_rf(trial, X_train, y_train, X_val, y_val):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 2, 20)
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

def objective_svc(trial, X_train, y_train, X_val, y_val):
    C = trial.suggest_float("C", 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly"])
    model = SVC(C=C, kernel=kernel, probability=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

In [444]:
def tune_model(model_name, objective_fn, X_train, y_train, X_val, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_fn(trial, X_train, y_train, X_val, y_val), n_trials=n_trials)

    console.print(f"[green]✅ Best {model_name} Params:[/green] {study.best_params}")
    console.print(f"[cyan]Best Validation Accuracy:[/cyan] {study.best_value:.4f}")

    # Re-train best model with best params
    if model_name == "KNN":
        best_model = KNeighborsClassifier(**study.best_params)
    elif model_name == "LogisticRegression":
        best_model = LogisticRegression(**study.best_params, max_iter=200)
    elif model_name == "RandomForest":
        best_model = RandomForestClassifier(**study.best_params, random_state=42)
    elif model_name == "SVC":
        best_model = SVC(**study.best_params, probability=True)

    best_model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]))

    return best_model, study.best_params, study.best_value

In [445]:
all_models = {
    "KNN": objective_knn,
    "LogisticRegression": objective_logreg,
    "RandomForest": objective_rf,
    "SVC": objective_svc
}

best_models = {}
results = []

for model_name, obj_fn in all_models.items():
    console.print(f"[bold yellow]🔎 Tuning {model_name}...[/bold yellow]")
    model, params, score = tune_model(model_name, obj_fn, X_train, y_train, X_val, y_val, n_trials=20)
    best_models[model_name] = model
    results.append({"Model": model_name, "Val Accuracy": score, "Params": params})

results_df = pd.DataFrame(results)
console.print(results_df)

🔎 Tuning KNN...

[I 2025-08-25 18:22:48,638] A new study created in memory with name: no-name-56b8b50f-01b9-4338-9dc2-32cd9b3d1e2c
[I 2025-08-25 18:22:48,647] Trial 0 finished with value: 1.0 and parameters: {'n_neighbors': 15}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:22:48,653] Trial 1 finished with value: 1.0 and parameters: {'n_neighbors': 6}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:22:48,661] Trial 2 finished with value: 1.0 and parameters: {'n_neighbors': 13}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:22:48,667] Trial 3 finished with value: 1.0 and parameters: {'n_neighbors': 19}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:22:48,674] Trial 4 finished with value: 1.0 and parameters: {'n_neighbors': 6}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:22:48,681] Trial 5 finished with value: 1.0 and parameters: {'n_neighbors': 10}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:22:48,687] Trial 6 finished with value: 1.0 and parameters: {'n_neighbors': 20}. Best

✅ Best KNN Params: {'n_neighbors': 15}

Best Validation Accuracy: 1.0000

🔎 Tuning LogisticRegression...

...

🔎 Tuning SVC...

[I 2025-08-25 18:23:05,615] A new study created in memory with name: no-name-3b407d18-6ffa-47e2-ad8a-88c8218cb7b1
[I 2025-08-25 18:23:05,639] Trial 0 finished with value: 1.0 and parameters: {'C': 0.47161208129672033, 'kernel': 'rbf'}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:23:05,662] Trial 1 finished with value: 0.9333333333333333 and parameters: {'C': 0.12253025289946524, 'kernel': 'poly'}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:23:05,676] Trial 2 finished with value: 1.0 and parameters: {'C': 0.27865671820171517, 'kernel': 'rbf'}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:23:05,708] Trial 3 finished with value: 1.0 and parameters: {'C': 0.4882611042377666, 'kernel': 'rbf'}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:23:05,733] Trial 4 finished with value: 0.6666666666666666 and parameters: {'C': 0.033624266107475674, 'kernel': 'rbf'}. Best is trial 0 with value: 1.0.
[I 2025-08-25 18:23:05,765] Trial 5 finished with value: 1.0 and parameters: {'C

✅ Best SVC Params: {'C': 0.47161208129672033, 'kernel': 'rbf'}

Best Validation Accuracy: 1.0000

Model  Val Accuracy  \
0                 KNN           1.0   
1  LogisticRegression           1.0   
2        RandomForest           1.0   
3                 SVC           1.0   

                                             Params  
0                               {'n_neighbors': 15}  
1  {'C': 0.5454085408151761, 'solver': 'liblinear'}  
2             {'n_estimators': 240, 'max_depth': 2}  
3       {'C': 0.47161208129672033, 'kernel': 'rbf'}

In [446]:
results_df = pd.DataFrame(results)
console.print(results_df)

Model  Val Accuracy  \
0                 KNN           1.0   
1  LogisticRegression           1.0   
2        RandomForest           1.0   
3                 SVC           1.0   

                                             Params  
0                               {'n_neighbors': 15}  
1  {'C': 0.5454085408151761, 'solver': 'liblinear'}  
2             {'n_estimators': 240, 'max_depth': 2}  
3       {'C': 0.47161208129672033, 'kernel': 'rbf'}

In [447]:
best_row = results_df.sort_values(by="Val Accuracy", ascending=False).iloc[0]
best_model_name = best_row["Model"]
best_model = best_models[best_model_name]

console.print(f"[bold green]🏆 Best Model: {best_model_name} with Val Accuracy {best_row['Val Accuracy']:.4f}[/bold green]")

save_model(best_model, f"models/{best_model_name}_best.pkl")

🏆 Best Model: KNN with Val Accuracy 1.0000

✅ Model saved at: models/KNN_best.pkl

In [448]:
def train_best_model(model_name, best_params, X_train, y_train, X_val, y_val):
    """
    Train the best model with Optuna-selected hyperparameters.
    Uses Train + Validation data together for final training.
    """
    # Merge train + validation
    X_train_full = pd.concat([X_train, X_val])
    y_train_full = pd.concat([y_train, y_val])

    if model_name == "KNN":
        model = KNeighborsClassifier(**best_params)
    elif model_name == "LogisticRegression":
        model = LogisticRegression(**best_params, max_iter=200)
    elif model_name == "RandomForest":
        model = RandomForestClassifier(**best_params, random_state=42)
    elif model_name == "SVC":
        model = SVC(**best_params, probability=True)
    else:
        raise ValueError(f"❌ Unsupported model type: {model_name}")

    model.fit(X_train_full, y_train_full)
    console.print(f"[green]✅ Final {model_name} trained with best parameters[/green]")
    return model

In [449]:
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, class_names=None):
    """Evaluate model on train, validation, and test sets with metrics + confusion matrix"""
    results = {}
    splits = {"Train": (X_train, y_train), "Validation": (X_val, y_val), "Test": (X_test, y_test)}

    table = Table(title="KNN Model Evaluation", show_lines=True, box=ROUNDED)
    table.add_column("Dataset", style="bold cyan")
    table.add_column("Accuracy", style="bold green")
    table.add_column("Precision", style="yellow")
    table.add_column("Recall", style="magenta")
    table.add_column("F1", style="red")
    for split_name, (X_split, y_split) in splits.items():
        y_pred = model.predict(X_split)

        acc = accuracy_score(y_split, y_pred)
        prec = precision_score(y_split, y_pred, average="weighted")
        rec = recall_score(y_split, y_pred, average="weighted")
        f1 = f1_score(y_split, y_pred, average="weighted")


        results[split_name] = {
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1": f1,
        }

        table.add_row(
            split_name,
            f"{acc:.3f}", f"{prec:.3f}", f"{rec:.3f}", f"{f1:.3f}",
        )

    console.print(table)

    # Confusion Matrix (Test Set)
    y_test_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_test_pred)
    if class_names is None:
        class_names = [str(i) for i in np.unique(y_test)]

    fig_cm = ff.create_annotated_heatmap(z=cm, x=class_names, y=class_names,
                                         colorscale="Blues", showscale=True)
    fig_cm.update_layout(title="Confusion Matrix (Test Set)", title_x=0.5)
    fig_cm.show()

    return results

In [450]:
def save_model(model, filepath="model/knn_iris.pkl"):
    """Save trained model"""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    joblib.dump(model, filepath)
    console.print(f"[green]💾 Model saved at:[/green] {filepath}")


In [451]:
def load_model(filepath="model/knn_iris.pkl"):
    """Load model"""
    if not os.path.exists(filepath):
        console.print(f"[red]❌ Model not found at {filepath}[/red]")
        return None
    model = joblib.load(filepath)
    console.print(f"[green]✅ Model loaded from:[/green] {filepath}")
    return model

In [452]:
# Get the best model from Optuna
best_model_row = results_df.sort_values(by="Val Accuracy", ascending=False).iloc[0]
best_model_name = best_model_row["Model"]
best_params = best_model_row["Params"]

# Train final model
final_model = train_best_model(best_model_name, best_params, X_train, y_train, X_val, y_val)

# Evaluate on Test Set
evaluate_model(final_model, X_train, y_train, X_val, y_val, X_test, y_test,
               class_names=["Setosa", "Versicolor", "Virginica"])

# Save the trained model
save_model(final_model, f"models/{best_model_name}_best_Iris.pkl")

✅ Final KNN trained with best parameters

                 KNN Model Evaluation                 
╭────────────┬──────────┬───────────┬────────┬───────╮
│ Dataset    │ Accuracy │ Precision │ Recall │ F1    │
├────────────┼──────────┼───────────┼────────┼───────┤
│ Train      │ 0.951    │ 0.951     │ 0.951  │ 0.951 │
├────────────┼──────────┼───────────┼────────┼───────┤
│ Validation │ 1.000    │ 1.000     │ 1.000  │ 1.000 │
├────────────┼──────────┼───────────┼────────┼───────┤
│ Test       │ 0.967    │ 0.970     │ 0.967  │ 0.967 │
╰────────────┴──────────┴───────────┴────────┴───────╯

💾 Model saved at: models/KNN_best_Iris.pkl

In [453]:
# Load model later
loaded_knn = load_model("models/KNN_best_Iris.pkl")

✅ Model loaded from: models/KNN_best_Iris.pkl